In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [4]:
import numpy as np
import plotly
import pandas as pd
import plotly.graph_objects as go
import geopandas as gpd
from scipy.stats.stats import pearsonr   

In [5]:
df = pd.read_csv(r"Unique stops - last attempt 2.csv")#, dtype={'Latitude': float, 'Longitude':float, 'Year':float, 'Search Conducted':str,'Search Disposition':str,'Search Outcome':str,'Search Reason':str,'Search Reason For Stop':str,'Search Type':str,'Search Arrest Reason':str, "Charge":str})

In [8]:
"""
coordinates_dict holds all citations, by latitude/longitude.
each long/lat entry of coordinates_dict holds a small list, which counts the number of observations for each race 
at that location.
coordinates_dict[longitude][latitude](race) = 
        # of observations of this race at this location[0,5], [sum(0-5)==all stops at this location]

each internal list is organized as follows:
[asian, black, hispanic, native american, other, white, TOTAL STOPS, %asian, %black, %hispanic, %native american, %other, %white]
"""
coordinates_dict = {}
for x in np.arange(-77.6,-76.5,0.005):
    coordinates_dict[np.around(x,4)] = {}

for x in coordinates_dict:
    for y in np.arange(38.5,39.5,0.005):
        coordinates_dict[x][np.around(y,4)] = {"Total Unique Stops":([0]*6), "Number of Citations":([0]*6), "Search Conducted":([0]*6), "Probable Cause":([0]*6), "Arrested":([0]*6), "Accidents":([0]*6),}


def rounder(x):#rounds to nearest .005
    return np.around(.005 * round(x/.005),4)

#for assigning to list in coordinates_dict
race_key_dict = {'ASIAN':0, 'BLACK':1, 'HISPANIC':2, 'NATIVE AMERICAN':3, 'OTHER':4, 'WHITE':5}

#assigning values to coordinate_dict's race-list
for i in range(len(df)):
    try:    
        coordinates_dict[rounder(df["Longitude"].iloc[i])][rounder(df["Latitude"].iloc[i])]["Total Unique Stops"][race_key_dict[df["Race"].iloc[i]]] += 1
        coordinates_dict[rounder(df["Longitude"].iloc[i])][rounder(df["Latitude"].iloc[i])]["Number of Citations"][race_key_dict[df["Race"].iloc[i]]] += df["Num_Citations"].iloc[i]
        if df["D_Arrested"].iloc[i]==1:
            coordinates_dict[rounder(df["Longitude"].iloc[i])][rounder(df["Latitude"].iloc[i])]["Arrested"][race_key_dict[df["Race"].iloc[i]]] += 1
        if df["D_SearchConducted"].iloc[i]==1:
            coordinates_dict[rounder(df["Longitude"].iloc[i])][rounder(df["Latitude"].iloc[i])]["Search Conducted"][race_key_dict[df["Race"].iloc[i]]] += 1
        if df["D_ProbableCause"].iloc[i]==1:
            coordinates_dict[rounder(df["Longitude"].iloc[i])][rounder(df["Latitude"].iloc[i])]["Probable Cause"][race_key_dict[df["Race"].iloc[i]]] += 1
        if df["D_Accident"].iloc[i]==1:
            coordinates_dict[rounder(df["Longitude"].iloc[i])][rounder(df["Latitude"].iloc[i])]["Accidents"][race_key_dict[df["Race"].iloc[i]]] += 1
    except:
        pass

for x in coordinates_dict:#longitude
    for y in coordinates_dict[x]:           #latitude
        for dicts in coordinates_dict[x][y]:
            coordinates_dict[x][y][dicts].append(sum(z for z in coordinates_dict[x][y][dicts]))    #append sum to end        

for x in coordinates_dict:
    for y in coordinates_dict[x]:
        for dicts in coordinates_dict[x][y]:
            if not all(z==0 for z in coordinates_dict[x][y][dicts]):
                for z in coordinates_dict[x][y][dicts][:6]:
    #                 print(z, coordinates_dict[x][y][6], z/coordinates_dict[x][y][6])
                    coordinates_dict[x][y][dicts].append(z/coordinates_dict[x][y][dicts][6])
            else:
                coordinates_dict[x][y][dicts].extend([0]*6)

In [9]:
# print(coordinates_dict[-77][38.985]["Number of Citations"][7])
# print([coordinates_dict[x][y]["Number of Citations"][7] for x in coordinates_dict for y in coordinates_dict[x]])

In [10]:
def divide(num, denom):
    if denom!=0:
        return num/denom
    else:
        return 0

In [11]:
lat = []
lon = []
for x in coordinates_dict:
    for y in coordinates_dict[x]:
        lon.append(x)
        lat.append(y)

df5 = pd.DataFrame(
{
    
     'Total Citations': [coordinates_dict[x][y]["Number of Citations"][6] for x in coordinates_dict for y in coordinates_dict[x]],
     'Asian Citations': [coordinates_dict[x][y]["Number of Citations"][7] for x in coordinates_dict for y in coordinates_dict[x]],
     'Black Citations': [coordinates_dict[x][y]["Number of Citations"][8] for x in coordinates_dict for y in coordinates_dict[x]],
     'Hispanic Citations': [coordinates_dict[x][y]["Number of Citations"][9] for x in coordinates_dict for y in coordinates_dict[x]],
     'Native American Citations': [coordinates_dict[x][y]["Number of Citations"][10] for x in coordinates_dict for y in coordinates_dict[x]],
     'Other Citations': [coordinates_dict[x][y]["Number of Citations"][11] for x in coordinates_dict for y in coordinates_dict[x]],
     'White Citations': [coordinates_dict[x][y]["Number of Citations"][12] for x in coordinates_dict for y in coordinates_dict[x]],
 
    
     'Total Arrested': [coordinates_dict[x][y]["Arrested"][6] for x in coordinates_dict for y in coordinates_dict[x]],
     'Asian Arrested': [coordinates_dict[x][y]["Arrested"][7] for x in coordinates_dict for y in coordinates_dict[x]],
     'Black Arrested': [coordinates_dict[x][y]["Arrested"][8] for x in coordinates_dict for y in coordinates_dict[x]],
     'Hispanic Arrested': [coordinates_dict[x][y]["Arrested"][9] for x in coordinates_dict for y in coordinates_dict[x]],
     'Native American Arrested': [coordinates_dict[x][y]["Arrested"][10] for x in coordinates_dict for y in coordinates_dict[x]],
     'Other Arrested': [coordinates_dict[x][y]["Arrested"][11] for x in coordinates_dict for y in coordinates_dict[x]],
     'White Arrested': [coordinates_dict[x][y]["Arrested"][12] for x in coordinates_dict for y in coordinates_dict[x]],
    
    
     'Total Arrested Per Stop': [divide(coordinates_dict[x][y]["Arrested"][6], coordinates_dict[x][y]["Total Unique Stops"][6]) for x in coordinates_dict for y in coordinates_dict[x]],
     'Asian Arrested Per Stop': [divide(coordinates_dict[x][y]["Arrested"][7], coordinates_dict[x][y]["Total Unique Stops"][7]) for x in coordinates_dict for y in coordinates_dict[x]],
     'Black Arrested Per Stop': [divide(coordinates_dict[x][y]["Arrested"][8], coordinates_dict[x][y]["Total Unique Stops"][8]) for x in coordinates_dict for y in coordinates_dict[x]],
     'Hispanic Arrested Per Stop': [divide(coordinates_dict[x][y]["Arrested"][9], coordinates_dict[x][y]["Total Unique Stops"][9]) for x in coordinates_dict for y in coordinates_dict[x]],
     'Native American Arrested Per Stop': [divide(coordinates_dict[x][y]["Arrested"][10], coordinates_dict[x][y]["Total Unique Stops"][10]) for x in coordinates_dict for y in coordinates_dict[x]],
     'Other Arrested Per Stop': [divide(coordinates_dict[x][y]["Arrested"][11], coordinates_dict[x][y]["Total Unique Stops"][11]) for x in coordinates_dict for y in coordinates_dict[x]],
     'White Arrested Per Stop': [divide(coordinates_dict[x][y]["Arrested"][12], coordinates_dict[x][y]["Total Unique Stops"][12]) for x in coordinates_dict for y in coordinates_dict[x]],

    
     'Total Citations Per Stop': [divide(coordinates_dict[x][y]["Number of Citations"][6], coordinates_dict[x][y]["Total Unique Stops"][6]) for x in coordinates_dict for y in coordinates_dict[x]],
     'Asian Citations Per Stop': [divide(coordinates_dict[x][y]["Number of Citations"][7], coordinates_dict[x][y]["Total Unique Stops"][7]) for x in coordinates_dict for y in coordinates_dict[x]],
     'Black Citations Per Stop': [divide(coordinates_dict[x][y]["Number of Citations"][8], coordinates_dict[x][y]["Total Unique Stops"][8]) for x in coordinates_dict for y in coordinates_dict[x]],
     'Hispanic Citations Per Stop': [divide(coordinates_dict[x][y]["Number of Citations"][9], coordinates_dict[x][y]["Total Unique Stops"][9]) for x in coordinates_dict for y in coordinates_dict[x]],
     'Native American Citations Per Stop': [divide(coordinates_dict[x][y]["Number of Citations"][10], coordinates_dict[x][y]["Total Unique Stops"][10]) for x in coordinates_dict for y in coordinates_dict[x]],
     'Other Citations Per Stop': [divide(coordinates_dict[x][y]["Number of Citations"][11], coordinates_dict[x][y]["Total Unique Stops"][11]) for x in coordinates_dict for y in coordinates_dict[x]],
     'White Citations Per Stop': [divide(coordinates_dict[x][y]["Number of Citations"][12], coordinates_dict[x][y]["Total Unique Stops"][12]) for x in coordinates_dict for y in coordinates_dict[x]],
    
    
     'Total Searched Per Stop': [divide(coordinates_dict[x][y]["Search Conducted"][6], coordinates_dict[x][y]["Total Unique Stops"][6]) for x in coordinates_dict for y in coordinates_dict[x]],
     'Asian Searched Per Stop': [divide(coordinates_dict[x][y]["Search Conducted"][7], coordinates_dict[x][y]["Total Unique Stops"][7]) for x in coordinates_dict for y in coordinates_dict[x]],
     'Black Searched Per Stop': [divide(coordinates_dict[x][y]["Search Conducted"][8], coordinates_dict[x][y]["Total Unique Stops"][8]) for x in coordinates_dict for y in coordinates_dict[x]],
     'Hispanic Searched Per Stop': [divide(coordinates_dict[x][y]["Search Conducted"][9], coordinates_dict[x][y]["Total Unique Stops"][9]) for x in coordinates_dict for y in coordinates_dict[x]],
     'Native American Searched Per Stop': [divide(coordinates_dict[x][y]["Search Conducted"][10], coordinates_dict[x][y]["Total Unique Stops"][10]) for x in coordinates_dict for y in coordinates_dict[x]],
     'Other Searched Per Stop': [divide(coordinates_dict[x][y]["Search Conducted"][11], coordinates_dict[x][y]["Total Unique Stops"][11]) for x in coordinates_dict for y in coordinates_dict[x]],
     'White Searched Per Stop': [divide(coordinates_dict[x][y]["Search Conducted"][12], coordinates_dict[x][y]["Total Unique Stops"][12]) for x in coordinates_dict for y in coordinates_dict[x]],

 
     'Total Probable Cause Per Stop': [divide(coordinates_dict[x][y]["Probable Cause"][6], coordinates_dict[x][y]["Total Unique Stops"][6]) for x in coordinates_dict for y in coordinates_dict[x]],
     'Asian Probable Cause Per Stop': [divide(coordinates_dict[x][y]["Probable Cause"][7], coordinates_dict[x][y]["Total Unique Stops"][7]) for x in coordinates_dict for y in coordinates_dict[x]],
     'Black Probable Cause Per Stop': [divide(coordinates_dict[x][y]["Probable Cause"][8], coordinates_dict[x][y]["Total Unique Stops"][8]) for x in coordinates_dict for y in coordinates_dict[x]],
     'Hispanic Probable Cause Per Stop': [divide(coordinates_dict[x][y]["Probable Cause"][9], coordinates_dict[x][y]["Total Unique Stops"][9]) for x in coordinates_dict for y in coordinates_dict[x]],
     'Native American Probable Cause Per Stop': [divide(coordinates_dict[x][y]["Probable Cause"][10], coordinates_dict[x][y]["Total Unique Stops"][10]) for x in coordinates_dict for y in coordinates_dict[x]],
     'Other Probable Cause Per Stop': [divide(coordinates_dict[x][y]["Probable Cause"][11], coordinates_dict[x][y]["Total Unique Stops"][11]) for x in coordinates_dict for y in coordinates_dict[x]],
     'White Probable Cause Per Stop': [divide(coordinates_dict[x][y]["Probable Cause"][12], coordinates_dict[x][y]["Total Unique Stops"][12]) for x in coordinates_dict for y in coordinates_dict[x]],       

    

     'Total Searched': [coordinates_dict[x][y]["Search Conducted"][6] for x in coordinates_dict for y in coordinates_dict[x]],
     'Asian Searched': [coordinates_dict[x][y]["Search Conducted"][7] for x in coordinates_dict for y in coordinates_dict[x]],
     'Black Searched': [coordinates_dict[x][y]["Search Conducted"][8] for x in coordinates_dict for y in coordinates_dict[x]],
     'Hispanic Searched': [coordinates_dict[x][y]["Search Conducted"][9] for x in coordinates_dict for y in coordinates_dict[x]],
     'Native American Searched': [coordinates_dict[x][y]["Search Conducted"][10] for x in coordinates_dict for y in coordinates_dict[x]],
     'Other Searched': [coordinates_dict[x][y]["Search Conducted"][11] for x in coordinates_dict for y in coordinates_dict[x]],
     'White Searched': [coordinates_dict[x][y]["Search Conducted"][12] for x in coordinates_dict for y in coordinates_dict[x]],


     'Total Stopped': [coordinates_dict[x][y]["Total Unique Stops"][6] for x in coordinates_dict for y in coordinates_dict[x]],
     'Asian Stopped': [coordinates_dict[x][y]["Total Unique Stops"][7] for x in coordinates_dict for y in coordinates_dict[x]],
     'Black Stopped': [coordinates_dict[x][y]["Total Unique Stops"][8] for x in coordinates_dict for y in coordinates_dict[x]],
     'Hispanic Stopped': [coordinates_dict[x][y]["Total Unique Stops"][9] for x in coordinates_dict for y in coordinates_dict[x]],
     'Native American Stopped': [coordinates_dict[x][y]["Total Unique Stops"][10] for x in coordinates_dict for y in coordinates_dict[x]],
     'Other Stopped': [coordinates_dict[x][y]["Total Unique Stops"][11] for x in coordinates_dict for y in coordinates_dict[x]],
     'White Stopped': [coordinates_dict[x][y]["Total Unique Stops"][12] for x in coordinates_dict for y in coordinates_dict[x]],       

    
     'Asian as Percent of Stopped 1': [divide(coordinates_dict[x][y]["Total Unique Stops"][7], coordinates_dict[x][y]["Total Unique Stops"][6]) for x in coordinates_dict for y in coordinates_dict[x]],
     'Black as Percent of Stopped 1': [divide(coordinates_dict[x][y]["Total Unique Stops"][8], coordinates_dict[x][y]["Total Unique Stops"][6]) for x in coordinates_dict for y in coordinates_dict[x]],
     'Hispanic as Percent of Stopped 1': [divide(coordinates_dict[x][y]["Total Unique Stops"][9], coordinates_dict[x][y]["Total Unique Stops"][6]) for x in coordinates_dict for y in coordinates_dict[x]],
     'Native American as Percent of Stopped 1': [divide(coordinates_dict[x][y]["Total Unique Stops"][10], coordinates_dict[x][y]["Total Unique Stops"][6]) for x in coordinates_dict for y in coordinates_dict[x]],
     'Other as Percent of Stopped 1': [divide(coordinates_dict[x][y]["Total Unique Stops"][11], coordinates_dict[x][y]["Total Unique Stops"][6]) for x in coordinates_dict for y in coordinates_dict[x]],
     'White as Percent of Stopped 1': [divide(coordinates_dict[x][y]["Total Unique Stops"][12], coordinates_dict[x][y]["Total Unique Stops"][6]) for x in coordinates_dict for y in coordinates_dict[x]],       

    
     'Asian as Percent of Stopped 2': [divide(coordinates_dict[x][y]["Total Unique Stops"][0], coordinates_dict[x][y]["Total Unique Stops"][6]) for x in coordinates_dict for y in coordinates_dict[x]],
     'Black as Percent of Stopped 2': [divide(coordinates_dict[x][y]["Total Unique Stops"][1], coordinates_dict[x][y]["Total Unique Stops"][6]) for x in coordinates_dict for y in coordinates_dict[x]],
     'Hispanic as Percent of Stopped 2': [divide(coordinates_dict[x][y]["Total Unique Stops"][2], coordinates_dict[x][y]["Total Unique Stops"][6]) for x in coordinates_dict for y in coordinates_dict[x]],
     'Native American as Percent of Stopped 2': [divide(coordinates_dict[x][y]["Total Unique Stops"][3], coordinates_dict[x][y]["Total Unique Stops"][6]) for x in coordinates_dict for y in coordinates_dict[x]],
     'Other as Percent of Stopped 2': [divide(coordinates_dict[x][y]["Total Unique Stops"][4], coordinates_dict[x][y]["Total Unique Stops"][6]) for x in coordinates_dict for y in coordinates_dict[x]],
     'White as Percent of Stopped 2': [divide(coordinates_dict[x][y]["Total Unique Stops"][5], coordinates_dict[x][y]["Total Unique Stops"][6]) for x in coordinates_dict for y in coordinates_dict[x]],       


     'Total Probable Cause': [coordinates_dict[x][y]["Probable Cause"][6] for x in coordinates_dict for y in coordinates_dict[x]],
     'Asian Probable Cause': [coordinates_dict[x][y]["Probable Cause"][7] for x in coordinates_dict for y in coordinates_dict[x]],
     'Black Probable Cause': [coordinates_dict[x][y]["Probable Cause"][8] for x in coordinates_dict for y in coordinates_dict[x]],
     'Hispanic Probable Cause': [coordinates_dict[x][y]["Probable Cause"][9] for x in coordinates_dict for y in coordinates_dict[x]],
     'Native American Probable Cause': [coordinates_dict[x][y]["Probable Cause"][10] for x in coordinates_dict for y in coordinates_dict[x]],
     'Other Probable Cause': [coordinates_dict[x][y]["Probable Cause"][11] for x in coordinates_dict for y in coordinates_dict[x]],
     'White Probable Cause': [coordinates_dict[x][y]["Probable Cause"][12] for x in coordinates_dict for y in coordinates_dict[x]],       

    
     'Total Accidents': [coordinates_dict[x][y]["Accidents"][6] for x in coordinates_dict for y in coordinates_dict[x]],
     'Asian Accidents': [coordinates_dict[x][y]["Accidents"][7] for x in coordinates_dict for y in coordinates_dict[x]],
     'Black Accidents': [coordinates_dict[x][y]["Accidents"][8] for x in coordinates_dict for y in coordinates_dict[x]],
     'Hispanic Accidents': [coordinates_dict[x][y]["Accidents"][9] for x in coordinates_dict for y in coordinates_dict[x]],
     'Native American Accidents': [coordinates_dict[x][y]["Accidents"][10] for x in coordinates_dict for y in coordinates_dict[x]],
     'Other Accidents': [coordinates_dict[x][y]["Accidents"][11] for x in coordinates_dict for y in coordinates_dict[x]],
     'White Accidents': [coordinates_dict[x][y]["Accidents"][12] for x in coordinates_dict for y in coordinates_dict[x]],       


     'Total Arrested Per Accident': [divide(coordinates_dict[x][y]["Arrested"][6], coordinates_dict[x][y]["Accidents"][6]) for x in coordinates_dict for y in coordinates_dict[x]],
     'Asian Arrested Per Accident': [divide(coordinates_dict[x][y]["Arrested"][7], coordinates_dict[x][y]["Accidents"][7]) for x in coordinates_dict for y in coordinates_dict[x]],
     'Black Arrested Per Accident': [divide(coordinates_dict[x][y]["Arrested"][8], coordinates_dict[x][y]["Accidents"][8]) for x in coordinates_dict for y in coordinates_dict[x]],
     'Hispanic Arrested Per Accident': [divide(coordinates_dict[x][y]["Arrested"][9], coordinates_dict[x][y]["Accidents"][9]) for x in coordinates_dict for y in coordinates_dict[x]],
     'Native American Arrested Per Accident': [divide(coordinates_dict[x][y]["Arrested"][10], coordinates_dict[x][y]["Accidents"][10]) for x in coordinates_dict for y in coordinates_dict[x]],
     'Other Arrested Per Accident': [divide(coordinates_dict[x][y]["Arrested"][11], coordinates_dict[x][y]["Accidents"][11]) for x in coordinates_dict for y in coordinates_dict[x]],
     'White Arrested Per Accident': [divide(coordinates_dict[x][y]["Arrested"][12], coordinates_dict[x][y]["Accidents"][12]) for x in coordinates_dict for y in coordinates_dict[x]],

    
     'Total Searched Per Accident': [divide(coordinates_dict[x][y]["Search Conducted"][6], coordinates_dict[x][y]["Accidents"][6]) for x in coordinates_dict for y in coordinates_dict[x]],
     'Asian Searched Per Accident': [divide(coordinates_dict[x][y]["Search Conducted"][7], coordinates_dict[x][y]["Accidents"][7]) for x in coordinates_dict for y in coordinates_dict[x]],
     'Black Searched Per Accident': [divide(coordinates_dict[x][y]["Search Conducted"][8], coordinates_dict[x][y]["Accidents"][8]) for x in coordinates_dict for y in coordinates_dict[x]],
     'Hispanic Searched Per Accident': [divide(coordinates_dict[x][y]["Search Conducted"][9], coordinates_dict[x][y]["Accidents"][9]) for x in coordinates_dict for y in coordinates_dict[x]],
     'Native American Searched Per Accident': [divide(coordinates_dict[x][y]["Search Conducted"][10], coordinates_dict[x][y]["Accidents"][10]) for x in coordinates_dict for y in coordinates_dict[x]],
     'Other Searched Per Accident': [divide(coordinates_dict[x][y]["Search Conducted"][11], coordinates_dict[x][y]["Accidents"][11]) for x in coordinates_dict for y in coordinates_dict[x]],
     'White Searched Per Accident': [divide(coordinates_dict[x][y]["Search Conducted"][12], coordinates_dict[x][y]["Accidents"][12]) for x in coordinates_dict for y in coordinates_dict[x]],

     'Latitude': lat,
     'Longitude': lon,

}
)


In [65]:
#  if coordinates_dict[x][y]['Total Unique Stops'][6]>0

"""
FILTERED WITH MIN CELL SIZE (0 STOPS) 
"""

citations_df = pd.DataFrame(
{
     'Total Citations': [coordinates_dict[x][y]["Number of Citations"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0], 
     'Asian Citations': [coordinates_dict[x][y]["Number of Citations"][7] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Black Citations': [coordinates_dict[x][y]["Number of Citations"][8] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Hispanic Citations': [coordinates_dict[x][y]["Number of Citations"][9] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Native American Citations': [coordinates_dict[x][y]["Number of Citations"][10] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Other Citations': [coordinates_dict[x][y]["Number of Citations"][11] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'White Citations': [coordinates_dict[x][y]["Number of Citations"][12] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0]
    })

arrests_df = pd.DataFrame(
{
     'Total Arrested': [coordinates_dict[x][y]["Arrested"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Asian Arrested Per Stop': [divide(coordinates_dict[x][y]["Arrested"][7], coordinates_dict[x][y]["Total Unique Stops"][7]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Black Arrested Per Stop': [divide(coordinates_dict[x][y]["Arrested"][8], coordinates_dict[x][y]["Total Unique Stops"][8]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Hispanic Arrested Per Stop': [divide(coordinates_dict[x][y]["Arrested"][9], coordinates_dict[x][y]["Total Unique Stops"][9]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Native American Arrested Per Stop': [divide(coordinates_dict[x][y]["Arrested"][10], coordinates_dict[x][y]["Total Unique Stops"][10]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Other Arrested Per Stop': [divide(coordinates_dict[x][y]["Arrested"][11], coordinates_dict[x][y]["Total Unique Stops"][11]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'White Arrested Per Stop': [divide(coordinates_dict[x][y]["Arrested"][12], coordinates_dict[x][y]["Total Unique Stops"][12]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0]
    
    })


searches_df = pd.DataFrame(
{    
     'Total Searched Per Stop': [divide(coordinates_dict[x][y]["Search Conducted"][6], coordinates_dict[x][y]["Total Unique Stops"][6]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Asian Searched Per Stop': [divide(coordinates_dict[x][y]["Search Conducted"][0], coordinates_dict[x][y]["Total Unique Stops"][0]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Black Searched Per Stop': [divide(coordinates_dict[x][y]["Search Conducted"][1], coordinates_dict[x][y]["Total Unique Stops"][1]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Hispanic Searched Per Stop': [divide(coordinates_dict[x][y]["Search Conducted"][2], coordinates_dict[x][y]["Total Unique Stops"][2]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Native American Searched Per Stop': [divide(coordinates_dict[x][y]["Search Conducted"][3], coordinates_dict[x][y]["Total Unique Stops"][3]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Other Searched Per Stop': [divide(coordinates_dict[x][y]["Search Conducted"][4], coordinates_dict[x][y]["Total Unique Stops"][4]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'White Searched Per Stop': [divide(coordinates_dict[x][y]["Search Conducted"][5], coordinates_dict[x][y]["Total Unique Stops"][5]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0]
    })

probable_cause_df = pd.DataFrame(
    {
     'Total Probable Cause': [coordinates_dict[x][y]["Probable Cause"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Asian Probable Cause': [coordinates_dict[x][y]["Probable Cause"][7] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Black Probable Cause': [coordinates_dict[x][y]["Probable Cause"][8] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Hispanic Probable Cause': [coordinates_dict[x][y]["Probable Cause"][9] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Native American Probable Cause': [coordinates_dict[x][y]["Probable Cause"][10] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Other Probable Cause': [coordinates_dict[x][y]["Probable Cause"][11] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'White Probable Cause': [coordinates_dict[x][y]["Probable Cause"][12] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0]
        
    })

probable_cause_df_2 = pd.DataFrame(
    {
     'Total Probable Cause': [divide(coordinates_dict[x][y]["Probable Cause"][6], coordinates_dict[x][y]["Total Unique Stops"][6]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Asian Probable Cause': [divide(coordinates_dict[x][y]["Probable Cause"][7], coordinates_dict[x][y]["Total Unique Stops"][6]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Black Probable Cause': [divide(coordinates_dict[x][y]["Probable Cause"][8], coordinates_dict[x][y]["Total Unique Stops"][6]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Hispanic Probable Cause': [divide(coordinates_dict[x][y]["Probable Cause"][9], coordinates_dict[x][y]["Total Unique Stops"][6]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Native American Probable Cause': [divide(coordinates_dict[x][y]["Probable Cause"][10], coordinates_dict[x][y]["Total Unique Stops"][6]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Other Probable Cause': [divide(coordinates_dict[x][y]["Probable Cause"][11], coordinates_dict[x][y]["Total Unique Stops"][6]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'White Probable Cause': [divide(coordinates_dict[x][y]["Probable Cause"][12], coordinates_dict[x][y]["Total Unique Stops"][6]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0]
        
    })


accidents_df = pd.DataFrame(
    {
     'Total Accidents': [coordinates_dict[x][y]["Accidents"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Asian Accidents': [coordinates_dict[x][y]["Accidents"][7] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Black Accidents': [coordinates_dict[x][y]["Accidents"][8] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Hispanic Accidents': [coordinates_dict[x][y]["Accidents"][9] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Native American Accidents': [coordinates_dict[x][y]["Accidents"][10] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Other Accidents': [coordinates_dict[x][y]["Accidents"][11] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'White Accidents': [coordinates_dict[x][y]["Accidents"][12] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0]   
    })

searches_per_accident_df = pd.DataFrame(
{
     'Total Searched Per Accident': [divide(coordinates_dict[x][y]["Search Conducted"][6], coordinates_dict[x][y]["Accidents"][6]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Asian Searched Per Accident': [divide(coordinates_dict[x][y]["Search Conducted"][0], coordinates_dict[x][y]["Accidents"][0]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Black Searched Per Accident': [divide(coordinates_dict[x][y]["Search Conducted"][1], coordinates_dict[x][y]["Accidents"][1]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Hispanic Searched Per Accident': [divide(coordinates_dict[x][y]["Search Conducted"][2], coordinates_dict[x][y]["Accidents"][2]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Native American Searched Per Accident': [divide(coordinates_dict[x][y]["Search Conducted"][3], coordinates_dict[x][y]["Accidents"][3]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Other Searched Per Accident': [divide(coordinates_dict[x][y]["Search Conducted"][4], coordinates_dict[x][y]["Accidents"][4]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'White Searched Per Accident': [divide(coordinates_dict[x][y]["Search Conducted"][5], coordinates_dict[x][y]["Accidents"][5]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0]

})
  
pct_stopped_df = pd.DataFrame({
    
     'Total as Percent of Stopped': [coordinates_dict[x][y]["Total Unique Stops"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Asian as Percent of Stopped': [divide(coordinates_dict[x][y]["Total Unique Stops"][0], coordinates_dict[x][y]["Total Unique Stops"][6]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Black as Percent of Stopped': [divide(coordinates_dict[x][y]["Total Unique Stops"][1], coordinates_dict[x][y]["Total Unique Stops"][6]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Hispanic as Percent of Stopped': [divide(coordinates_dict[x][y]["Total Unique Stops"][2], coordinates_dict[x][y]["Total Unique Stops"][6]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Native American as Percent of Stopped': [divide(coordinates_dict[x][y]["Total Unique Stops"][3], coordinates_dict[x][y]["Total Unique Stops"][6]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Other as Percent of Stopped': [divide(coordinates_dict[x][y]["Total Unique Stops"][4], coordinates_dict[x][y]["Total Unique Stops"][6]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'White as Percent of Stopped': [divide(coordinates_dict[x][y]["Total Unique Stops"][5], coordinates_dict[x][y]["Total Unique Stops"][6]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0]       

})

pct_stopped_df_2 = pd.DataFrame({
    
     'Total as Percent of Stopped': [1 for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Asian as Percent of Stopped': [coordinates_dict[x][y]["Total Unique Stops"][7] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Black as Percent of Stopped': [coordinates_dict[x][y]["Total Unique Stops"][8] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Hispanic as Percent of Stopped': [coordinates_dict[x][y]["Total Unique Stops"][9] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Native American as Percent of Stopped': [coordinates_dict[x][y]["Total Unique Stops"][10] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Other as Percent of Stopped': [coordinates_dict[x][y]["Total Unique Stops"][11] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'White as Percent of Stopped': [coordinates_dict[x][y]["Total Unique Stops"][12] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0]       

})

raw_total_stops = pd.DataFrame({
    
     'Total as Percent of Stopped': [1 for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Asian as Percent of Stopped': [coordinates_dict[x][y]["Total Unique Stops"][0] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Black as Percent of Stopped': [coordinates_dict[x][y]["Total Unique Stops"][1] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Hispanic as Percent of Stopped': [coordinates_dict[x][y]["Total Unique Stops"][2] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Native American as Percent of Stopped': [coordinates_dict[x][y]["Total Unique Stops"][3] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Other as Percent of Stopped': [coordinates_dict[x][y]["Total Unique Stops"][4] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'White as Percent of Stopped': [coordinates_dict[x][y]["Total Unique Stops"][5] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0]       

})

In [60]:
share_of_citations_df = pd.DataFrame(
{
     'Total Citations': [coordinates_dict[x][y]["Number of Citations"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0], 
     'Asian Citations': [coordinates_dict[x][y]["Number of Citations"][7] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Black Citations': [coordinates_dict[x][y]["Number of Citations"][8] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Hispanic Citations': [coordinates_dict[x][y]["Number of Citations"][9] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Native American Citations': [coordinates_dict[x][y]["Number of Citations"][10] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Other Citations': [coordinates_dict[x][y]["Number of Citations"][11] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'White Citations': [coordinates_dict[x][y]["Number of Citations"][12] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0]
    })

total_citations_df = pd.DataFrame(
{
     'Total Citations': [coordinates_dict[x][y]["Number of Citations"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0], 
     'Asian Citations': [coordinates_dict[x][y]["Number of Citations"][0] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Black Citations': [coordinates_dict[x][y]["Number of Citations"][1] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Hispanic Citations': [coordinates_dict[x][y]["Number of Citations"][2] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Native American Citations': [coordinates_dict[x][y]["Number of Citations"][3] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Other Citations': [coordinates_dict[x][y]["Number of Citations"][4] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'White Citations': [coordinates_dict[x][y]["Number of Citations"][5] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0]
    })


arrests_share_div_stops_share_df = pd.DataFrame(
{
     'Total Arrested': [coordinates_dict[x][y]["Arrested"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Asian Arrested Per Stop': [divide(coordinates_dict[x][y]["Arrested"][7], coordinates_dict[x][y]["Total Unique Stops"][7]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Black Arrested Per Stop': [divide(coordinates_dict[x][y]["Arrested"][8], coordinates_dict[x][y]["Total Unique Stops"][8]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Hispanic Arrested Per Stop': [divide(coordinates_dict[x][y]["Arrested"][9], coordinates_dict[x][y]["Total Unique Stops"][9]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Native American Arrested Per Stop': [divide(coordinates_dict[x][y]["Arrested"][10], coordinates_dict[x][y]["Total Unique Stops"][10]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Other Arrested Per Stop': [divide(coordinates_dict[x][y]["Arrested"][11], coordinates_dict[x][y]["Total Unique Stops"][11]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'White Arrested Per Stop': [divide(coordinates_dict[x][y]["Arrested"][12], coordinates_dict[x][y]["Total Unique Stops"][12]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0]
    
    })

arrests_share_div_stops_total_df = pd.DataFrame(
{
     'Total Arrested': [coordinates_dict[x][y]["Arrested"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Asian Arrested Per Stop': [divide(coordinates_dict[x][y]["Arrested"][7], coordinates_dict[x][y]["Total Unique Stops"][0]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Black Arrested Per Stop': [divide(coordinates_dict[x][y]["Arrested"][8], coordinates_dict[x][y]["Total Unique Stops"][1]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Hispanic Arrested Per Stop': [divide(coordinates_dict[x][y]["Arrested"][9], coordinates_dict[x][y]["Total Unique Stops"][2]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Native American Arrested Per Stop': [divide(coordinates_dict[x][y]["Arrested"][10], coordinates_dict[x][y]["Total Unique Stops"][3]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Other Arrested Per Stop': [divide(coordinates_dict[x][y]["Arrested"][11], coordinates_dict[x][y]["Total Unique Stops"][4]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'White Arrested Per Stop': [divide(coordinates_dict[x][y]["Arrested"][12], coordinates_dict[x][y]["Total Unique Stops"][5]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0]
    
    })


arrests_total_div_stops_share_df = pd.DataFrame(
{
     'Total Arrested': [coordinates_dict[x][y]["Arrested"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Asian Arrested Per Stop': [divide(coordinates_dict[x][y]["Arrested"][0], coordinates_dict[x][y]["Total Unique Stops"][7]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Black Arrested Per Stop': [divide(coordinates_dict[x][y]["Arrested"][1], coordinates_dict[x][y]["Total Unique Stops"][8]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Hispanic Arrested Per Stop': [divide(coordinates_dict[x][y]["Arrested"][2], coordinates_dict[x][y]["Total Unique Stops"][9]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Native American Arrested Per Stop': [divide(coordinates_dict[x][y]["Arrested"][3], coordinates_dict[x][y]["Total Unique Stops"][10]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Other Arrested Per Stop': [divide(coordinates_dict[x][y]["Arrested"][4], coordinates_dict[x][y]["Total Unique Stops"][11]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'White Arrested Per Stop': [divide(coordinates_dict[x][y]["Arrested"][5], coordinates_dict[x][y]["Total Unique Stops"][12]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0]
    
    })

arrests_total_div_stops_total_df = pd.DataFrame(
{
     'Total Arrested': [coordinates_dict[x][y]["Arrested"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Asian Arrested Per Stop': [divide(coordinates_dict[x][y]["Arrested"][0], coordinates_dict[x][y]["Total Unique Stops"][0]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Black Arrested Per Stop': [divide(coordinates_dict[x][y]["Arrested"][1], coordinates_dict[x][y]["Total Unique Stops"][1]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Hispanic Arrested Per Stop': [divide(coordinates_dict[x][y]["Arrested"][2], coordinates_dict[x][y]["Total Unique Stops"][2]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Native American Arrested Per Stop': [divide(coordinates_dict[x][y]["Arrested"][3], coordinates_dict[x][y]["Total Unique Stops"][3]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Other Arrested Per Stop': [divide(coordinates_dict[x][y]["Arrested"][4], coordinates_dict[x][y]["Total Unique Stops"][4]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'White Arrested Per Stop': [divide(coordinates_dict[x][y]["Arrested"][5], coordinates_dict[x][y]["Total Unique Stops"][5]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0]
    
    })

raw_stops_df = pd.DataFrame({
    
     'Total Stopped': [1 for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Asian Stopped': [coordinates_dict[x][y]["Total Unique Stops"][0] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Black Stopped': [coordinates_dict[x][y]["Total Unique Stops"][1] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Hispanic Stopped': [coordinates_dict[x][y]["Total Unique Stops"][2] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Native American Stopped': [coordinates_dict[x][y]["Total Unique Stops"][3] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Other Stopped': [coordinates_dict[x][y]["Total Unique Stops"][4] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'White Stopped': [coordinates_dict[x][y]["Total Unique Stops"][5] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0]       

})

arrests_div_stops_df = pd.DataFrame(
{
     'Total Arrested': [coordinates_dict[x][y]["Arrested"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Asian Arrested Per Stop': [divide(coordinates_dict[x][y]["Arrested"][0], coordinates_dict[x][y]["Total Unique Stops"][0]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Black Arrested Per Stop': [divide(coordinates_dict[x][y]["Arrested"][1], coordinates_dict[x][y]["Total Unique Stops"][1]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Hispanic Arrested Per Stop': [divide(coordinates_dict[x][y]["Arrested"][2], coordinates_dict[x][y]["Total Unique Stops"][2]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Native American Arrested Per Stop': [divide(coordinates_dict[x][y]["Arrested"][3], coordinates_dict[x][y]["Total Unique Stops"][3]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Other Arrested Per Stop': [divide(coordinates_dict[x][y]["Arrested"][4], coordinates_dict[x][y]["Total Unique Stops"][4]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'White Arrested Per Stop': [divide(coordinates_dict[x][y]["Arrested"][5], coordinates_dict[x][y]["Total Unique Stops"][5]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0]
    
    })


arrests_race_div_arrests_total_df = pd.DataFrame(
{
     'Total Arrested': [coordinates_dict[x][y]["Arrested"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Asian Arrested Per Stop': [divide(coordinates_dict[x][y]["Arrested"][0], coordinates_dict[x][y]["Arrested"][6]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Black Arrested Per Stop': [divide(coordinates_dict[x][y]["Arrested"][1], coordinates_dict[x][y]["Arrested"][6]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Hispanic Arrested Per Stop': [divide(coordinates_dict[x][y]["Arrested"][2], coordinates_dict[x][y]["Arrested"][6]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Native American Arrested Per Stop': [divide(coordinates_dict[x][y]["Arrested"][3], coordinates_dict[x][y]["Arrested"][6]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Other Arrested Per Stop': [divide(coordinates_dict[x][y]["Arrested"][4], coordinates_dict[x][y]["Arrested"][6]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'White Arrested Per Stop': [divide(coordinates_dict[x][y]["Arrested"][5], coordinates_dict[x][y]["Arrested"][6]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0]
    
    })

arrests_race_div_arrests_total_df_2 = pd.DataFrame(
{
     'Total Arrested': [coordinates_dict[x][y]["Arrested"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Asian Arrested Per Stop': [coordinates_dict[x][y]["Arrested"][7] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Black Arrested Per Stop': [coordinates_dict[x][y]["Arrested"][8] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Hispanic Arrested Per Stop': [coordinates_dict[x][y]["Arrested"][9] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Native American Arrested Per Stop': [coordinates_dict[x][y]["Arrested"][10] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Other Arrested Per Stop': [coordinates_dict[x][y]["Arrested"][11] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'White Arrested Per Stop': [coordinates_dict[x][y]["Arrested"][12] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0]
    
    })


probable_cause_per_arrest_df = pd.DataFrame({

     'Total Prob Cause': [coordinates_dict[x][y]["Probable Cause"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Asian Prob Cause': [divide(coordinates_dict[x][y]["Probable Cause"][0], coordinates_dict[x][y]["Arrested"][6]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Black Prob Cause': [divide(coordinates_dict[x][y]["Probable Cause"][1], coordinates_dict[x][y]["Arrested"][6]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Hispanic Prob Cause': [divide(coordinates_dict[x][y]["Probable Cause"][2], coordinates_dict[x][y]["Arrested"][6]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Native American Prob Cause': [divide(coordinates_dict[x][y]["Probable Cause"][3], coordinates_dict[x][y]["Arrested"][6]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'Other Prob Cause': [divide(coordinates_dict[x][y]["Probable Cause"][4], coordinates_dict[x][y]["Arrested"][6]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0],
     'White Prob Cause': [divide(coordinates_dict[x][y]["Probable Cause"][5], coordinates_dict[x][y]["Total Unique Stops"][6]) for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>0]
})

In [157]:
"""
Creating df's weighted by total stops in that cell

"""

arrest_share_weighted_by_stops_df = pd.DataFrame(
{
     'Total Arrested': [item for sublist in [[coordinates_dict[x][y]["Arrested"][6]]*coordinates_dict[x][y]["Total Unique Stops"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>500] for item in sublist],
     'Asian Arrested Per Stop': [item for sublist in [[coordinates_dict[x][y]["Arrested"][7]]*coordinates_dict[x][y]["Total Unique Stops"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>500] for item in sublist],
     'Black Arrested Per Stop': [item for sublist in [[coordinates_dict[x][y]["Arrested"][8]]*coordinates_dict[x][y]["Total Unique Stops"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>500] for item in sublist],
     'Hispanic Arrested Per Stop': [item for sublist in [[coordinates_dict[x][y]["Arrested"][9]]*coordinates_dict[x][y]["Total Unique Stops"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>500] for item in sublist],
     'Native American Arrested Per Stop': [item for sublist in [[coordinates_dict[x][y]["Arrested"][10]]*coordinates_dict[x][y]["Total Unique Stops"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>500] for item in sublist],
     'Other Arrested Per Stop': [item for sublist in [[coordinates_dict[x][y]["Arrested"][11]]*coordinates_dict[x][y]["Total Unique Stops"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>500] for item in sublist],
     'White Arrested Per Stop': [item for sublist in [[coordinates_dict[x][y]["Arrested"][12]]*coordinates_dict[x][y]["Total Unique Stops"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>500] for item in sublist]    
})

arrest_share_weighted_by_arrests_df = pd.DataFrame(
{
     'Total Arrested': [item for sublist in [[coordinates_dict[x][y]["Arrested"][6]]*coordinates_dict[x][y]["Arrested"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>500] for item in sublist],
     'Asian Arrested Per Stop': [item for sublist in [[coordinates_dict[x][y]["Arrested"][7]]*coordinates_dict[x][y]["Arrested"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>500] for item in sublist],
     'Black Arrested Per Stop': [item for sublist in [[coordinates_dict[x][y]["Arrested"][8]]*coordinates_dict[x][y]["Arrested"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>500] for item in sublist],
     'Hispanic Arrested Per Stop': [item for sublist in [[coordinates_dict[x][y]["Arrested"][9]]*coordinates_dict[x][y]["Arrested"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>500] for item in sublist],
     'Native American Arrested Per Stop': [item for sublist in [[coordinates_dict[x][y]["Arrested"][10]]*coordinates_dict[x][y]["Arrested"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>500] for item in sublist],
     'Other Arrested Per Stop': [item for sublist in [[coordinates_dict[x][y]["Arrested"][11]]*coordinates_dict[x][y]["Arrested"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>500] for item in sublist],
     'White Arrested Per Stop': [item for sublist in [[coordinates_dict[x][y]["Arrested"][12]]*coordinates_dict[x][y]["Arrested"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>500] for item in sublist]    
})


probable_cause_weighted_by_stops = pd.DataFrame(
{
     'Total Prob Cause Searched': [item for sublist in [[coordinates_dict[x][y]["Probable Cause"][6]]*coordinates_dict[x][y]["Total Unique Stops"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>500] for item in sublist],
     'Asian Prob Cause Searched': [item for sublist in [[coordinates_dict[x][y]["Probable Cause"][7]]*coordinates_dict[x][y]["Total Unique Stops"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>500] for item in sublist],
     'Black Prob Cause Searched': [item for sublist in [[coordinates_dict[x][y]["Probable Cause"][8]]*coordinates_dict[x][y]["Total Unique Stops"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>500] for item in sublist],
     'Hispanic Prob Cause Searched': [item for sublist in [[coordinates_dict[x][y]["Probable Cause"][9]]*coordinates_dict[x][y]["Total Unique Stops"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>500] for item in sublist],
     'Native American Prob Cause Searched': [item for sublist in [[coordinates_dict[x][y]["Probable Cause"][10]]*coordinates_dict[x][y]["Total Unique Stops"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>500] for item in sublist],
     'Other Prob Cause Searched': [item for sublist in [[coordinates_dict[x][y]["Probable Cause"][11]]*coordinates_dict[x][y]["Total Unique Stops"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>500] for item in sublist],
     'White Prob Cause Searched': [item for sublist in [[coordinates_dict[x][y]["Probable Cause"][12]]*coordinates_dict[x][y]["Total Unique Stops"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>500] for item in sublist]    
})

probable_cause_weighted_by_prob_cause = pd.DataFrame(
{
     'Total Prob Cause Searched': [item for sublist in [[coordinates_dict[x][y]["Probable Cause"][6]]*coordinates_dict[x][y]["Probable Cause"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>500] for item in sublist],
     'Asian Prob Cause Searched': [item for sublist in [[coordinates_dict[x][y]["Probable Cause"][7]]*coordinates_dict[x][y]["Probable Cause"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>500] for item in sublist],
     'Black Prob Cause Searched': [item for sublist in [[coordinates_dict[x][y]["Probable Cause"][8]]*coordinates_dict[x][y]["Probable Cause"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>500] for item in sublist],
     'Hispanic Prob Cause Searched': [item for sublist in [[coordinates_dict[x][y]["Probable Cause"][9]]*coordinates_dict[x][y]["Probable Cause"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>500] for item in sublist],
     'Native American Prob Cause Searched': [item for sublist in [[coordinates_dict[x][y]["Probable Cause"][10]]*coordinates_dict[x][y]["Probable Cause"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>500] for item in sublist],
     'Other Prob Cause Searched': [item for sublist in [[coordinates_dict[x][y]["Probable Cause"][11]]*coordinates_dict[x][y]["Probable Cause"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>500] for item in sublist],
     'White Prob Cause Searched': [item for sublist in [[coordinates_dict[x][y]["Probable Cause"][12]]*coordinates_dict[x][y]["Probable Cause"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>500] for item in sublist]    
})


stops_weighted_by_stops = pd.DataFrame(
{
     'Total Unique Stops': [item for sublist in [[coordinates_dict[x][y]["Total Unique Stops"][6]]*coordinates_dict[x][y]["Total Unique Stops"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>500] for item in sublist],
     'Asian Unique Stops': [item for sublist in [[coordinates_dict[x][y]["Total Unique Stops"][7]]*coordinates_dict[x][y]["Total Unique Stops"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>500] for item in sublist],
     'Black Unique Stops': [item for sublist in [[coordinates_dict[x][y]["Total Unique Stops"][8]]*coordinates_dict[x][y]["Total Unique Stops"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>500] for item in sublist],
     'Hispanic Unique Stops': [item for sublist in [[coordinates_dict[x][y]["Total Unique Stops"][9]]*coordinates_dict[x][y]["Total Unique Stops"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>500] for item in sublist],
     'Native Unique Stops': [item for sublist in [[coordinates_dict[x][y]["Total Unique Stops"][10]]*coordinates_dict[x][y]["Total Unique Stops"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>500] for item in sublist],
     'Other Unique Stops': [item for sublist in [[coordinates_dict[x][y]["Total Unique Stops"][11]]*coordinates_dict[x][y]["Total Unique Stops"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>500] for item in sublist],
     'White Unique Stops': [item for sublist in [[coordinates_dict[x][y]["Total Unique Stops"][12]]*coordinates_dict[x][y]["Total Unique Stops"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>500] for item in sublist]
})


stops_weighted_by_prob_cause = pd.DataFrame(
{
     'Total Unique Stops': [item for sublist in [[coordinates_dict[x][y]["Total Unique Stops"][6]]*coordinates_dict[x][y]["Probable Cause"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>500] for item in sublist],
     'Asian Unique Stops': [item for sublist in [[coordinates_dict[x][y]["Total Unique Stops"][7]]*coordinates_dict[x][y]["Probable Cause"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>500] for item in sublist],
     'Black Unique Stops': [item for sublist in [[coordinates_dict[x][y]["Total Unique Stops"][8]]*coordinates_dict[x][y]["Probable Cause"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>500] for item in sublist],
     'Hispanic Unique Stops': [item for sublist in [[coordinates_dict[x][y]["Total Unique Stops"][9]]*coordinates_dict[x][y]["Probable Cause"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>500] for item in sublist],
     'Native Unique Stops': [item for sublist in [[coordinates_dict[x][y]["Total Unique Stops"][10]]*coordinates_dict[x][y]["Probable Cause"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>500] for item in sublist],
     'Other Unique Stops': [item for sublist in [[coordinates_dict[x][y]["Total Unique Stops"][11]]*coordinates_dict[x][y]["Probable Cause"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>500] for item in sublist],
     'White Unique Stops': [item for sublist in [[coordinates_dict[x][y]["Total Unique Stops"][12]]*coordinates_dict[x][y]["Probable Cause"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>500] for item in sublist]
})


stops_weighted_by_arrest = pd.DataFrame(
{
     'Total Unique Stops': [item for sublist in [[coordinates_dict[x][y]["Total Unique Stops"][6]]*coordinates_dict[x][y]["Arrested"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>500] for item in sublist],
     'Asian Unique Stops': [item for sublist in [[coordinates_dict[x][y]["Total Unique Stops"][7]]*coordinates_dict[x][y]["Arrested"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>500] for item in sublist],
     'Black Unique Stops': [item for sublist in [[coordinates_dict[x][y]["Total Unique Stops"][8]]*coordinates_dict[x][y]["Arrested"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>500] for item in sublist],
     'Hispanic Unique Stops': [item for sublist in [[coordinates_dict[x][y]["Total Unique Stops"][9]]*coordinates_dict[x][y]["Arrested"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>500] for item in sublist],
     'Native Unique Stops': [item for sublist in [[coordinates_dict[x][y]["Total Unique Stops"][10]]*coordinates_dict[x][y]["Arrested"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>500] for item in sublist],
     'Other Unique Stops': [item for sublist in [[coordinates_dict[x][y]["Total Unique Stops"][11]]*coordinates_dict[x][y]["Arrested"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>500] for item in sublist],
     'White Unique Stops': [item for sublist in [[coordinates_dict[x][y]["Total Unique Stops"][12]]*coordinates_dict[x][y]["Arrested"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>500] for item in sublist]
})


arrests_share_div_stops_share_weighted_by_stops_df = pd.DataFrame({
    
     'Total Arrested': [item for sublist in [[coordinates_dict[x][y]["Arrested"][6]]*coordinates_dict[x][y]["Total Unique Stops"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>500] for item in sublist],
     'Asian Arrested Per Stop': [item for sublist in [[divide(coordinates_dict[x][y]["Arrested"][7], coordinates_dict[x][y]["Total Unique Stops"][7])]*coordinates_dict[x][y]["Total Unique Stops"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>500] for item in sublist],
     'Black Arrested Per Stop': [item for sublist in [[divide(coordinates_dict[x][y]["Arrested"][8], coordinates_dict[x][y]["Total Unique Stops"][8])]*coordinates_dict[x][y]["Total Unique Stops"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>500] for item in sublist],
     'Hispanic Arrested Per Stop': [item for sublist in [[divide(coordinates_dict[x][y]["Arrested"][9], coordinates_dict[x][y]["Total Unique Stops"][9])]*coordinates_dict[x][y]["Total Unique Stops"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>500] for item in sublist],
     'Native American Arrested Per Stop': [item for sublist in [[divide(coordinates_dict[x][y]["Arrested"][10], coordinates_dict[x][y]["Total Unique Stops"][10])]*coordinates_dict[x][y]["Total Unique Stops"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>500] for item in sublist],
     'Other Arrested Per Stop': [item for sublist in [[divide(coordinates_dict[x][y]["Arrested"][11], coordinates_dict[x][y]["Total Unique Stops"][11])]*coordinates_dict[x][y]["Total Unique Stops"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>500] for item in sublist],
     'White Arrested Per Stop': [item for sublist in [[divide(coordinates_dict[x][y]["Arrested"][12], coordinates_dict[x][y]["Total Unique Stops"][12])]*coordinates_dict[x][y]["Total Unique Stops"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>500] for item in sublist]    

})


probable_cause_per_arrest_weighted_by_stops_df = pd.DataFrame({

    
     'Total Arrested': [item for sublist in [[coordinates_dict[x][y]["Arrested"][6]]*coordinates_dict[x][y]["Total Unique Stops"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>500] for item in sublist],
     'Asian Arrested Per Stop': [item for sublist in [[divide(coordinates_dict[x][y]["Probable Cause"][0], coordinates_dict[x][y]["Arrested"][6])]*coordinates_dict[x][y]["Total Unique Stops"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>500] for item in sublist],
     'Black Arrested Per Stop': [item for sublist in [[divide(coordinates_dict[x][y]["Probable Cause"][1], coordinates_dict[x][y]["Arrested"][6])]*coordinates_dict[x][y]["Total Unique Stops"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>500] for item in sublist],
     'Hispanic Arrested Per Stop': [item for sublist in [[divide(coordinates_dict[x][y]["Probable Cause"][2], coordinates_dict[x][y]["Arrested"][6])]*coordinates_dict[x][y]["Total Unique Stops"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>500] for item in sublist],
     'Native American Arrested Per Stop': [item for sublist in [[divide(coordinates_dict[x][y]["Probable Cause"][3], coordinates_dict[x][y]["Arrested"][6])]*coordinates_dict[x][y]["Total Unique Stops"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>500] for item in sublist],
     'Other Arrested Per Stop': [item for sublist in [[divide(coordinates_dict[x][y]["Probable Cause"][4], coordinates_dict[x][y]["Arrested"][6])]*coordinates_dict[x][y]["Total Unique Stops"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>500] for item in sublist],
     'White Arrested Per Stop': [item for sublist in [[divide(coordinates_dict[x][y]["Probable Cause"][5], coordinates_dict[x][y]["Arrested"][6])]*coordinates_dict[x][y]["Total Unique Stops"][6] for x in coordinates_dict for y in coordinates_dict[x] if coordinates_dict[x][y]['Total Unique Stops'][6]>500] for item in sublist]    
    

})

In [129]:
def create_corr_matrix(df1, df2, df2_var, df1_name, df2_verb):
    test = np.zeros((7,7))
    for i, col in enumerate(test):
        for j, row in enumerate(test[i]):
            test[i][j] = round(pearsonr(df1.iloc[:, i], df2.iloc[:, j])[0], 2)


    corr_df = pd.DataFrame(test)

    corr_df.columns = df1.columns
    
    corr_df.rename({0:f'Total {df2_var}', 1: f'Asian {df2_var}', 2: f'Black {df2_var}', 3: f'Hispanic {df2_var}', 4: f'Native American {df2_var}', 5: f'Other {df2_var}', 6: f'White {df2_var}'}, inplace=True)
    corr_df.iloc[0,0] = np.nan
#     corr_df[list(df.index)[0]].iloc[0] = np.nan

#     corr_df['Total Citations'].iloc[0] = np.nan


    import seaborn as sns
    cm = sns.light_palette("green", as_cmap=True)


    with pd.option_context('display.precision', 2):
        html1 = (corr_df.style
                .background_gradient(cmap=cm, axis=None, high=0.2)
               )

    html1.columns.name = f'Correlation Matrix: {df1_name} vs. {df2_verb}'
    return(html1)
#     return(corr_df)

In [139]:
def create_corr_matrix_dataframe(df1, df2, df2_var, df1_name, df2_verb):
    test = np.zeros((7,7))
    for i, col in enumerate(test):
        for j, row in enumerate(test[i]):
            test[i][j] = round(pearsonr(df1.iloc[:, i], df2.iloc[:, j])[0], 2)


    corr_df = pd.DataFrame(test)

    corr_df.columns = df1.columns
    
    corr_df.rename({0:f'Total {df2_var}', 1: f'Asian {df2_var}', 2: f'Black {df2_var}', 3: f'Hispanic {df2_var}', 4: f'Native American {df2_var}', 5: f'Other {df2_var}', 6: f'White {df2_var}'}, inplace=True)
    corr_df.iloc[0] = np.nan
    #     corr_df[list(df.index)[0]].iloc[0] = np.nan

    #     corr_df['Total Citations'].iloc[0] = np.nan


#     import seaborn as sns
#     cm = sns.light_palette("green", as_cmap=True)


#     with pd.option_context('display.precision', 2):
#         html1 = (corr_df.style
#                 .background_gradient(cmap=cm, axis=None, high=0.2)
#                )

#     html1.columns.name = f'Correlation Matrix: {df1_name} vs. {df2_verb}'
#     return(html1)
    return(corr_df)

In [83]:
def create_corr_matrix_2(df1, df2, df1_name, df2_var):
    test = np.zeros((7,7))
    for i, col in enumerate(test):
        for j, row in enumerate(test[i]):
            test[i][j] = round(pearsonr([x for t, x in enumerate(df1.iloc[:, i]) if x != 0 and df2.iloc[:, j][t] != 0], [x for b,x in enumerate(df2.iloc[:, j]) if x!= 0 and df1.iloc[:, i][b] != 0])[0], 2)


    corr_df = pd.DataFrame(test)

    corr_df.columns = df1.columns
    
    corr_df.rename({0:f'Total {df2_var}', 1: f'Asian {df2_var}', 2: f'Black {df2_var}', 3: f'Hispanic {df2_var}', 4: f'Native American {df2_var}', 5: f'Other {df2_var}', 6: f'White {df2_var}'}, inplace=True)
#     corr_df['Total Citations'].iloc[0] = np.nan


    import seaborn as sns
    cm = sns.light_palette("blue", as_cmap=True)


    with pd.option_context('display.precision', 2):
        html1 = (corr_df.style
                .background_gradient(cmap=cm, axis=None, high=0.2)
               )

    html1.columns.name = f'Correlation Matrix: {df1_name} vs. {df2_var}'
    return(html1)

In [158]:
"""
We can see that the 
"""

create_corr_matrix(pct_stopped_df, arrests_df, "Arrested", "Percent of Stops", "Arrests")

Correlation Matrix: Percent of Stops vs. Arrests,Total as Percent of Stopped,Asian as Percent of Stopped,Black as Percent of Stopped,Hispanic as Percent of Stopped,Native American as Percent of Stopped,Other as Percent of Stopped,White as Percent of Stopped
Total Arrested,nan,0.08,0.26,0.19,0.03,0.10,0.27
Asian Arrested,-0.01,0.01,0.02,0.02,0.01,0.03,0.02
Black Arrested,0.08,0.03,0.10,0.06,-0.00,0.03,0.09
Hispanic Arrested,0.06,0.03,0.07,0.06,0.00,0.04,0.07
Native American Arrested,-0.00,-0.00,-0.01,-0.01,0.00,0.00,-0.01
Other Arrested,-0.00,0.01,0.02,0.02,0.00,0.01,0.02
White Arrested,-0.10,-0.05,-0.14,-0.10,-0.01,-0.06,-0.13


In [172]:
"""
Same as above but colors are for each column. We can see that Black Arrested, and then Hispanic Arrested, are the darkest in every zone. White Arrested is the lightest in each zone
"""

create_corr_matrix(stops_weighted_by_stops, arrests_share_div_stops_share_weighted_by_stops_df, "Share Arrested / Share Stopped", "Percent of Stops", "% Arrests / %Stops")

Correlation Matrix: Percent of Stops vs. % Arrests / %Stops,Total Unique Stops,Asian Unique Stops,Black Unique Stops,Hispanic Unique Stops,Native Unique Stops,Other Unique Stops,White Unique Stops
Total Share Arrested / Share Stopped,nan,0.03,-0.03,-0.00,0.03,-0.02,-0.01
Asian Share Arrested / Share Stopped,-0.21,0.00,0.12,0.04,-0.06,0.02,0.00
Black Share Arrested / Share Stopped,0.21,0.04,-0.07,-0.07,-0.03,-0.03,0.03
Hispanic Share Arrested / Share Stopped,0.20,-0.04,-0.04,-0.06,0.06,0.03,0.10
Native American Share Arrested / Share Stopped,0.02,-0.00,0.07,-0.07,0.03,-0.04,0.06
Other Share Arrested / Share Stopped,-0.22,-0.07,0.16,-0.04,0.12,-0.02,0.06
White Share Arrested / Share Stopped,-0.24,0.00,0.04,0.10,-0.02,0.00,-0.09


In [168]:
"""
CURRENTLY WORKING ON THIS

pct_stopped_df = divide(coordinates_dict[x][y]["Total Unique Stops"][0], coordinates_dict[x][y]["Total Unique Stops"][6])

arrests_df = divide(coordinates_dict[x][y]["Arrested"][7], coordinates_dict[x][y]["Total Unique Stops"][7])

        = (#black arrests / #total arrests)
          ________________________________
          
          (#black stops / #total stops)



why not (#black arrests / #black stops).


Is it wrong that a row is all positive or negative??

"""

create_corr_matrix(pct_stopped_df, arrests_df, "Arrested", "Percent of Stops", "Arrests")

Correlation Matrix: Percent of Stops vs. Arrests,Total as Percent of Stopped,Asian as Percent of Stopped,Black as Percent of Stopped,Hispanic as Percent of Stopped,Native American as Percent of Stopped,Other as Percent of Stopped,White as Percent of Stopped
Total Arrested,nan,0.08,0.26,0.19,0.03,0.10,0.27
Asian Arrested,-0.01,0.01,0.02,0.02,0.01,0.03,0.02
Black Arrested,0.08,0.03,0.10,0.06,-0.00,0.03,0.09
Hispanic Arrested,0.06,0.03,0.07,0.06,0.00,0.04,0.07
Native American Arrested,-0.00,-0.00,-0.01,-0.01,0.00,0.00,-0.01
Other Arrested,-0.00,0.01,0.02,0.02,0.00,0.01,0.02
White Arrested,-0.10,-0.05,-0.14,-0.10,-0.01,-0.06,-0.13


In [160]:
create_corr_matrix(pct_stopped_df, probable_cause_per_arrest_df, "Probable Cause Searched", "Percent of Stops", "Probable Cause Searched")

Correlation Matrix: Percent of Stops vs. Probable Cause Searched,Total as Percent of Stopped,Asian as Percent of Stopped,Black as Percent of Stopped,Hispanic as Percent of Stopped,Native American as Percent of Stopped,Other as Percent of Stopped,White as Percent of Stopped
Total Probable Cause Searched,nan,0.11,0.34,0.23,0.02,0.11,-0.02
Asian Probable Cause Searched,-0.02,0.02,-0.01,0.01,-0.01,0.01,-0.03
Black Probable Cause Searched,0.14,0.02,0.20,0.07,0.02,0.02,-0.05
Hispanic Probable Cause Searched,0.10,0.01,0.09,0.14,0.02,0.04,-0.04
Native American Probable Cause Searched,-0.00,-0.00,-0.01,-0.01,0.02,0.01,-0.00
Other Probable Cause Searched,-0.00,0.02,0.01,0.01,0.01,0.02,-0.03
White Probable Cause Searched,-0.17,-0.04,-0.21,-0.15,-0.03,-0.06,0.08


In [117]:
create_corr_matrix(pct_stopped_df, citations_df, "citations", 'pct stopped', 'cited')

Correlation Matrix: pct stopped vs. cited,Total as Percent of Stopped,Asian as Percent of Stopped,Black as Percent of Stopped,Hispanic as Percent of Stopped,Native American as Percent of Stopped,Other as Percent of Stopped,White as Percent of Stopped
Total citations,nan,0.01,0.14,0.12,-0.00,0.01,-0.07
Asian citations,-0.00,0.63,-0.07,-0.06,-0.01,0.01,-0.09
Black citations,0.09,-0.10,0.67,-0.02,0.02,-0.07,-0.42
Hispanic citations,0.08,-0.08,-0.05,0.67,-0.00,-0.06,-0.30
Native American citations,-0.00,-0.01,-0.02,-0.03,0.45,-0.02,-0.04
Other citations,0.00,0.01,-0.05,-0.03,-0.00,0.58,-0.14
White citations,-0.12,-0.11,-0.45,-0.36,-0.05,-0.13,0.61


In [132]:
create_corr_matrix_dataframe(stops_weighted_by_stops, arrests_share_div_stops_share_weighted_by_stops_df, 'arrest share / stops share', 'pct stopped', 'a').describe()

,Total Unique Stops,Asian Unique Stops,Black Unique Stops,Hispanic Unique Stops,Native Unique Stops,Other Unique Stops,White Unique Stops
count,6.000000,7.000000e+00,7.000000,7.000000,7.000000,7.000000e+00,7.000000
mean,-0.023333,9.912706e-19,0.030000,0.001429,0.010000,-9.912706e-19,0.017143
std,0.200466,2.309401e-02,0.044347,0.025448,0.022361,2.943920e-02,0.081591
min,-0.240000,-3.000000e-02,-0.040000,-0.030000,-0.030000,-3.000000e-02,-0.140000
25%,-0.175000,-1.500000e-02,0.020000,-0.010000,0.000000,-1.500000e-02,-0.005000
50%,-0.070000,0.000000e+00,0.030000,0.000000,0.020000,-1.000000e-02,0.030000
75%,0.162500,1.500000e-02,0.035000,0.005000,0.020000,5.000000e-03,0.065000
max,0.210000,3.000000e-02,0.110000,0.050000,0.040000,6.000000e-02,0.110000


In [134]:
create_corr_matrix(stops_weighted_by_stops, arrests_share_div_stops_share_weighted_by_stops_df, 'arrest share / stops share', 'pct stopped', 'a')

Correlation Matrix: pct stopped vs. a,Total Unique Stops,Asian Unique Stops,Black Unique Stops,Hispanic Unique Stops,Native Unique Stops,Other Unique Stops,White Unique Stops
Total arrest share / stops share,nan,0.02,0.04,0.05,0.02,0.00,0.05
Asian arrest share / stops share,-0.18,-0.00,0.03,0.00,0.00,0.01,-0.02
Black arrest share / stops share,0.21,0.03,0.01,0.00,-0.03,-0.01,0.08
Hispanic arrest share / stops share,0.21,-0.03,0.03,-0.00,0.02,0.06,0.11
Native American arrest share / stops share,0.02,0.01,0.03,-0.02,0.02,-0.02,0.03
Other arrest share / stops share,-0.16,-0.03,0.11,-0.03,0.04,-0.01,0.01
White arrest share / stops share,-0.24,-0.00,-0.04,0.01,0.00,-0.03,-0.14


In [140]:
create_corr_matrix_dataframe(stops_weighted_by_stops, arrests_share_div_stops_share_weighted_by_stops_df, 'arrest share / stops share', 'pct stopped', 'a').describe()

,Total Unique Stops,Asian Unique Stops,Black Unique Stops,Hispanic Unique Stops,Native Unique Stops,Other Unique Stops,White Unique Stops
count,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000
mean,-0.020000,-0.003333,0.021667,-0.006667,0.006667,0.003333,0.011667
std,0.195959,0.018619,0.054924,0.016330,0.023381,0.036697,0.096626
min,-0.240000,-0.020000,-0.070000,-0.030000,-0.030000,-0.040000,-0.150000
25%,-0.162500,-0.017500,0.012500,-0.017500,0.000000,-0.010000,-0.012500
50%,-0.065000,-0.005000,0.025000,-0.005000,0.005000,-0.005000,0.010000
75%,0.160000,0.000000,0.037500,0.007500,0.017500,0.007500,0.077500
max,0.210000,0.030000,0.100000,0.010000,0.040000,0.070000,0.120000


In [142]:
create_corr_matrix(stops_weighted_by_stops, arrests_share_div_stops_share_weighted_by_stops_df, 'arrest share / stops share', 'pct stopped', 'a')

Correlation Matrix: pct stopped vs. a,Total Unique Stops,Asian Unique Stops,Black Unique Stops,Hispanic Unique Stops,Native Unique Stops,Other Unique Stops,White Unique Stops
Total arrest share / stops share,nan,0.03,0.06,0.06,0.02,0.01,0.07
Asian arrest share / stops share,-0.17,-0.00,0.02,-0.00,0.00,0.01,-0.02
Black arrest share / stops share,0.21,0.03,0.03,0.01,-0.03,-0.00,0.10
Hispanic arrest share / stops share,0.21,-0.02,0.04,0.01,0.02,0.07,0.12
Native American arrest share / stops share,0.01,0.00,0.01,-0.02,0.01,-0.01,0.01
Other arrest share / stops share,-0.14,-0.02,0.10,-0.03,0.04,-0.01,0.01
White arrest share / stops share,-0.24,-0.01,-0.07,-0.01,0.00,-0.04,-0.15


In [144]:
create_corr_matrix(stops_weighted_by_stops, arrests_share_div_stops_share_weighted_by_stops_df, 'arrests share / stops share', 'weighted stops', 'arrests')

Correlation Matrix: weighted stops vs. arrests,Total Unique Stops,Asian Unique Stops,Black Unique Stops,Hispanic Unique Stops,Native Unique Stops,Other Unique Stops,White Unique Stops
Total arrests share / stops share,nan,0.03,0.06,0.06,0.02,0.01,0.07
Asian arrests share / stops share,-0.17,-0.00,0.02,-0.00,0.00,0.01,-0.02
Black arrests share / stops share,0.21,0.03,0.03,0.01,-0.03,-0.00,0.10
Hispanic arrests share / stops share,0.21,-0.02,0.04,0.01,0.02,0.07,0.12
Native American arrests share / stops share,0.01,0.00,0.01,-0.02,0.01,-0.01,0.01
Other arrests share / stops share,-0.14,-0.02,0.10,-0.03,0.04,-0.01,0.01
White arrests share / stops share,-0.24,-0.01,-0.07,-0.01,0.00,-0.04,-0.15


In [91]:
create_corr_matrix(stops_weighted_by_stops, arrest_share_weighted_by_stops_df, "Weighted Stops", "Weighted Arrests", "Arrested")

Correlation Matrix: Weighted Arrests vs. Arrested,Total Unique Stops,Asian Unique Stops,Black Unique Stops,Hispanic Unique Stops,Native Unique Stops,Other Unique Stops,White Unique Stops
Total Weighted Stops,nan,-0.02,0.18,0.14,0.01,-0.01,-0.03
Asian Weighted Stops,-0.18,0.18,-0.19,-0.13,-0.00,0.05,0.12
Black Weighted Stops,0.21,-0.04,0.53,0.05,-0.03,-0.03,-0.23
Hispanic Weighted Stops,0.21,-0.06,0.08,0.46,0.03,0.04,-0.16
Native American Weighted Stops,0.02,0.01,0.02,0.01,0.07,-0.02,0.02
Other Weighted Stops,-0.16,0.03,-0.02,-0.06,0.04,0.10,0.02
White Weighted Stops,-0.24,0.03,-0.44,-0.29,0.00,-0.02,0.26


In [97]:
create_corr_matrix(stops_weighted_by_stops, arrests_share_div_stops_share_weighted_by_stops_df, "Arrests Share / Stops Share", "Share of Stops", "Arrested")

Correlation Matrix: Share of Stops vs. Arrested,Total Unique Stops,Asian Unique Stops,Black Unique Stops,Hispanic Unique Stops,Native Unique Stops,Other Unique Stops,White Unique Stops
Total Arrests Share / Stops Share,nan,0.02,0.04,0.05,0.02,0.00,0.05
Asian Arrests Share / Stops Share,-0.18,-0.00,0.03,0.00,0.00,0.01,-0.02
Black Arrests Share / Stops Share,0.21,0.03,0.01,0.00,-0.03,-0.01,0.08
Hispanic Arrests Share / Stops Share,0.21,-0.03,0.03,-0.00,0.02,0.06,0.11
Native American Arrests Share / Stops Share,0.02,0.01,0.03,-0.02,0.02,-0.02,0.03
Other Arrests Share / Stops Share,-0.16,-0.03,0.11,-0.03,0.04,-0.01,0.01
White Arrests Share / Stops Share,-0.24,-0.00,-0.04,0.01,0.00,-0.03,-0.14


In [99]:
create_corr_matrix(stops_weighted_by_stops, probable_cause_weighted_by_stops, "Probable Cause Share", "Share of Stops", "Arrested")

Correlation Matrix: Share of Stops vs. Arrested,Total Unique Stops,Asian Unique Stops,Black Unique Stops,Hispanic Unique Stops,Native Unique Stops,Other Unique Stops,White Unique Stops
Total Probable Cause Share,nan,-0.01,0.26,0.10,-0.02,-0.03,-0.08
Asian Probable Cause Share,-0.30,0.07,-0.23,-0.07,-0.03,0.03,0.11
Black Probable Cause Share,0.40,-0.04,0.55,0.03,0.00,-0.09,-0.20
Hispanic Probable Cause Share,0.29,-0.02,0.12,0.42,0.05,0.02,-0.12
Native American Probable Cause Share,0.05,0.06,0.04,-0.00,0.09,-0.03,0.01
Other Probable Cause Share,-0.20,0.04,-0.06,0.03,0.01,0.03,0.03
White Probable Cause Share,-0.42,0.02,-0.47,-0.28,-0.03,0.05,0.21


In [95]:
create_corr_matrix(stops_weighted_by_prob_cause, probable_cause_weighted_by_prob_cause, "Probable Cause Share", "Share of Stops", "Arrested")

Correlation Matrix: Share of Stops vs. Arrested,Total Unique Stops,Asian Unique Stops,Black Unique Stops,Hispanic Unique Stops,Native Unique Stops,Other Unique Stops,White Unique Stops
Total Probable Cause Share,nan,-0.02,0.18,0.03,-0.05,-0.07,-0.19
Asian Probable Cause Share,-0.34,0.14,-0.30,-0.03,-0.04,0.09,0.29
Black Probable Cause Share,0.32,-0.11,0.58,-0.20,-0.02,-0.13,-0.42
Hispanic Probable Cause Share,0.17,-0.06,-0.16,0.50,0.02,-0.00,-0.20
Native American Probable Cause Share,0.02,0.01,-0.01,-0.02,0.27,0.01,-0.01
Other Probable Cause Share,-0.32,0.08,-0.21,0.04,0.05,0.13,0.13
White Probable Cause Share,-0.35,0.12,-0.39,-0.17,0.01,0.10,0.51


In [ ]:

arrest_share_weighted_by_arrests_df, 

In [100]:
#probable cause per arrest

create_corr_matrix(pct_stopped_df, probable_cause_per_arrest_df, "Prob Cause Searches per Arrest","Share of Stops", "Arrested")

Correlation Matrix: Share of Stops vs. Arrested,Total as Percent of Stopped,Asian as Percent of Stopped,Black as Percent of Stopped,Hispanic as Percent of Stopped,Native American as Percent of Stopped,Other as Percent of Stopped,White as Percent of Stopped
Total Prob Cause Searches per Arrest,nan,0.11,0.34,0.23,0.02,0.11,-0.02
Asian Prob Cause Searches per Arrest,-0.02,0.02,-0.01,0.01,-0.01,0.01,-0.03
Black Prob Cause Searches per Arrest,0.14,0.02,0.20,0.07,0.02,0.02,-0.05
Hispanic Prob Cause Searches per Arrest,0.10,0.01,0.09,0.14,0.02,0.04,-0.04
Native American Prob Cause Searches per Arrest,-0.00,-0.00,-0.01,-0.01,0.02,0.01,-0.00
Other Prob Cause Searches per Arrest,-0.00,0.02,0.01,0.01,0.01,0.02,-0.03
White Prob Cause Searches per Arrest,-0.17,-0.04,-0.21,-0.15,-0.03,-0.06,0.08


In [94]:
#probable cause per arrest, weighted by number of stops


create_corr_matrix(stops_weighted_by_stops, probable_cause_per_arrest_weighted_by_stops_df, "Prob Cause Searches per Arrest","Share of Stops", "Arrested")

Correlation Matrix: Share of Stops vs. Arrested,Total Unique Stops,Asian Unique Stops,Black Unique Stops,Hispanic Unique Stops,Native Unique Stops,Other Unique Stops,White Unique Stops
Total Prob Cause Searches per Arrest,nan,-0.02,0.12,0.02,-0.02,-0.04,-0.10
Asian Prob Cause Searches per Arrest,-0.18,0.05,-0.14,-0.03,-0.02,0.03,0.07
Black Prob Cause Searches per Arrest,0.21,-0.03,0.34,0.01,-0.01,-0.03,-0.15
Hispanic Prob Cause Searches per Arrest,0.21,-0.01,0.09,0.32,0.04,0.06,-0.09
Native American Prob Cause Searches per Arrest,0.02,-0.01,0.03,-0.00,0.09,-0.00,-0.00
Other Prob Cause Searches per Arrest,-0.16,0.07,0.01,0.06,0.02,0.06,0.06
White Prob Cause Searches per Arrest,-0.24,0.01,-0.31,-0.21,-0.02,-0.03,0.15


In [39]:
# create_corr_matrix(share_of_citations_df, arrests_race_div_arrests_total_df, "As share of all arrests", "Share of Citations", "Arrested")
create_corr_matrix_2(share_of_citations_df, arrests_race_div_arrests_total_df, "As share of all arrests", "Share of Citations")

Correlation Matrix: As share of all arrests vs. Share of Citations,Total Citations,Asian Citations,Black Citations,Hispanic Citations,Native American Citations,Other Citations,White Citations
Total Share of Citations,nan,-0.32,-0.22,-0.22,-0.45,-0.34,-0.08
Asian Share of Citations,-0.11,0.64,0.12,0.17,0.65,0.38,0.18
Black Share of Citations,0.03,-0.09,0.34,-0.34,-0.60,-0.21,0.13
Hispanic Share of Citations,0.09,-0.34,-0.11,0.37,-0.03,-0.24,0.10
Native American Share of Citations,-0.04,0.60,0.13,0.14,0.68,0.55,0.11
Other Share of Citations,-0.11,0.38,0.06,0.21,0.35,0.59,0.35
White Share of Citations,-0.11,0.29,-0.12,0.02,0.17,0.22,0.38


In [29]:
create_corr_matrix_2(share_of_citations_df, arrests_total_div_stops_total_df, "Arrests Share / Stops Share", "Share of Citations")

Correlation Matrix: Arrests Share / Stops Share vs. Share of Citations,Total Citations,Asian Citations,Black Citations,Hispanic Citations,Native American Citations,Other Citations,White Citations
Total Share of Citations,nan,-0.11,-0.10,-0.08,-0.53,-0.15,-0.06
Asian Share of Citations,-0.11,0.25,0.18,0.24,0.69,0.19,0.06
Black Share of Citations,0.03,0.20,0.40,-0.04,-0.31,0.10,0.23
Hispanic Share of Citations,0.09,0.20,0.12,0.32,-0.22,-0.03,0.25
Native American Share of Citations,-0.04,0.49,0.20,0.04,0.52,0.92,0.08
Other Share of Citations,-0.11,0.05,0.04,0.28,0.15,0.42,0.27
White Share of Citations,-0.11,-0.12,0.12,-0.00,0.23,-0.06,0.32


In [30]:
create_corr_matrix(share_of_citations_df, arrests_total_div_stops_total_df, "Arrests / Stops ", "Share of Citations", "Arrested")

Correlation Matrix: Share of Citations vs. Arrested,Total Citations,Asian Citations,Black Citations,Hispanic Citations,Native American Citations,Other Citations,White Citations
Total Arrests / Stops,nan,0.10,0.04,0.07,0.07,0.05,0.05
Asian Arrests / Stops,-0.01,0.02,-0.03,-0.02,-0.01,0.00,-0.03
Black Arrests / Stops,0.07,0.05,0.18,0.01,0.00,0.03,0.02
Hispanic Arrests / Stops,0.09,0.05,0.02,0.17,0.03,0.02,0.01
Native American Arrests / Stops,-0.00,-0.00,-0.01,-0.01,0.06,0.12,-0.01
Other Arrests / Stops,-0.01,-0.01,-0.03,0.00,0.00,0.05,-0.02
White Arrests / Stops,-0.06,-0.05,-0.08,-0.07,-0.01,-0.04,0.02


In [49]:
create_corr_matrix(pct_stopped_df, arrests_share_div_stops_share_df, "Arrests Share / Stops Share", "Share of Citations", "Arrested")
# create_corr_matrix_2(share_of_citations_df, arrests_share_div_stops_share_df, "Arrests Share / Stops Share", "Share of Citations")

Correlation Matrix: Share of Citations vs. Arrested,Total as Percent of Stopped,Asian as Percent of Stopped,Black as Percent of Stopped,Hispanic as Percent of Stopped,Native American as Percent of Stopped,Other as Percent of Stopped,White as Percent of Stopped
Total Arrests Share / Stops Share,nan,nan,nan,nan,nan,nan,nan
Asian Arrests Share / Stops Share,-0.01,0.01,0.02,0.02,0.01,0.03,0.02
Black Arrests Share / Stops Share,0.08,0.03,0.10,0.06,-0.00,0.03,0.09
Hispanic Arrests Share / Stops Share,0.06,0.03,0.07,0.06,0.00,0.04,0.07
Native American Arrests Share / Stops Share,-0.00,-0.00,-0.01,-0.01,0.00,0.00,-0.01
Other Arrests Share / Stops Share,-0.00,0.01,0.02,0.02,0.00,0.01,0.02
White Arrests Share / Stops Share,-0.10,-0.05,-0.14,-0.10,-0.01,-0.06,-0.13


In [40]:
create_corr_matrix(pct_stopped_df, arrests_share_div_stops_share_df, "Arrests Share / Stops Share", "Share of Citations", "Arrested")

Correlation Matrix: Share of Citations vs. Arrested,Total as Percent of Stopped,Asian as Percent of Stopped,Black as Percent of Stopped,Hispanic as Percent of Stopped,Native American as Percent of Stopped,Other as Percent of Stopped,White as Percent of Stopped
Total Arrests Share / Stops Share,nan,nan,nan,nan,nan,nan,nan
Asian Arrests Share / Stops Share,-0.01,0.01,0.02,0.02,0.01,0.03,0.02
Black Arrests Share / Stops Share,0.08,0.03,0.10,0.06,-0.00,0.03,0.09
Hispanic Arrests Share / Stops Share,0.06,0.03,0.07,0.06,0.00,0.04,0.07
Native American Arrests Share / Stops Share,-0.00,-0.00,-0.01,-0.01,0.00,0.00,-0.01
Other Arrests Share / Stops Share,-0.00,0.01,0.02,0.02,0.00,0.01,0.02
White Arrests Share / Stops Share,-0.10,-0.05,-0.14,-0.10,-0.01,-0.06,-0.13


In [41]:
create_corr_matrix_2(pct_stopped_df, arrests_share_div_stops_share_df, "Arrests Share / Stops Share", "Share of Citations")

Correlation Matrix: Arrests Share / Stops Share vs. Share of Citations,Total as Percent of Stopped,Asian as Percent of Stopped,Black as Percent of Stopped,Hispanic as Percent of Stopped,Native American as Percent of Stopped,Other as Percent of Stopped,White as Percent of Stopped
Total Share of Citations,nan,nan,nan,nan,nan,nan,nan
Asian Share of Citations,-0.09,-0.08,0.26,0.26,0.69,0.38,0.13
Black Share of Citations,0.09,-0.09,-0.44,-0.27,-0.56,-0.16,0.04
Hispanic Share of Citations,0.08,-0.05,-0.16,-0.35,-0.52,-0.16,0.01
Native American Share of Citations,-0.09,0.19,0.22,0.33,-0.15,0.19,0.18
Other Share of Citations,-0.09,0.25,0.17,0.18,0.02,-0.03,0.18
White Share of Citations,-0.14,0.10,0.46,0.46,0.53,0.24,-0.06


In [70]:
create_corr_matrix(share_of_citations_df, arrests_share_div_stops_total_df, "Arrests Share / Stops Total", "Share of Citations", "Arrested")

Correlation Matrix: Share of Citations vs. Arrested,Total Citations,Asian Citations,Black Citations,Hispanic Citations,Native American Citations,Other Citations,White Citations
Total Arrests Share / Stops Total Per Stop,nan,-0.01,-0.02,-0.02,0.00,-0.01,-0.02
Asian Arrests Share / Stops Total Per Stop,-0.01,0.04,-0.02,-0.02,-0.00,0.00,-0.03
Black Arrests Share / Stops Total Per Stop,0.07,0.03,0.15,-0.02,-0.01,0.01,0.00
Hispanic Arrests Share / Stops Total Per Stop,0.09,0.02,-0.01,0.15,0.02,0.00,-0.02
Native American Arrests Share / Stops Total Per Stop,-0.00,0.00,-0.01,-0.01,0.06,0.15,-0.01
Other Arrests Share / Stops Total Per Stop,-0.01,-0.00,-0.03,0.01,0.00,0.06,-0.02
White Arrests Share / Stops Total Per Stop,-0.06,-0.03,-0.07,-0.06,-0.00,-0.03,0.03


In [71]:
create_corr_matrix(share_of_citations_df, arrests_total_div_stops_share_df, "Arrests Total / Stops Share", "Share of Citations", "Arrested")

Correlation Matrix: Share of Citations vs. Arrested,Total Citations,Asian Citations,Black Citations,Hispanic Citations,Native American Citations,Other Citations,White Citations
Total Arrests Total / Stops Share Per Stop,nan,0.77,0.88,0.85,0.38,0.77,0.87
Asian Arrests Total / Stops Share Per Stop,-0.01,-0.00,-0.01,-0.01,-0.00,-0.01,-0.01
Black Arrests Total / Stops Share Per Stop,0.07,0.05,0.09,0.05,0.01,0.06,0.06
Hispanic Arrests Total / Stops Share Per Stop,0.09,0.06,0.10,0.08,0.03,0.09,0.08
Native American Arrests Total / Stops Share Per Stop,-0.00,-0.00,-0.00,-0.00,0.00,0.00,-0.00
Other Arrests Total / Stops Share Per Stop,-0.01,-0.01,-0.01,-0.01,0.00,-0.00,-0.01
White Arrests Total / Stops Share Per Stop,-0.06,-0.04,-0.07,-0.05,-0.01,-0.05,-0.05


In [72]:
create_corr_matrix(share_of_citations_df, arrests_total_div_stops_total_df, "Arrests Total / Stops Total", "Share of Citations", "Arrested")

Correlation Matrix: Share of Citations vs. Arrested,Total Citations,Asian Citations,Black Citations,Hispanic Citations,Native American Citations,Other Citations,White Citations
Total Arrests Total / Stops Total Per Stop,nan,0.10,0.04,0.07,0.07,0.05,0.05
Asian Arrests Total / Stops Total Per Stop,-0.01,0.02,-0.03,-0.02,-0.01,0.00,-0.03
Black Arrests Total / Stops Total Per Stop,0.07,0.05,0.18,0.01,0.00,0.03,0.02
Hispanic Arrests Total / Stops Total Per Stop,0.09,0.05,0.02,0.17,0.03,0.02,0.01
Native American Arrests Total / Stops Total Per Stop,-0.00,-0.00,-0.01,-0.01,0.06,0.12,-0.01
Other Arrests Total / Stops Total Per Stop,-0.01,-0.01,-0.03,0.00,0.00,0.05,-0.02
White Arrests Total / Stops Total Per Stop,-0.06,-0.05,-0.08,-0.07,-0.01,-0.04,0.02


In [61]:
create_corr_matrix(total_citations_df, arrests_share_div_stops_share_df, "Arrests Share / Stops Share", "Total Citations", "Arrested")

Correlation Matrix: Total Citations vs. Arrested,Total Citations,Asian Citations,Black Citations,Hispanic Citations,Native American Citations,Other Citations,White Citations
Total Arrests Share / Stops Share Per Stop,0.89,0.07,0.18,0.14,0.02,0.07,0.20
Asian Arrests Share / Stops Share Per Stop,0.71,0.09,0.23,0.18,0.02,0.08,0.24
Black Arrests Share / Stops Share Per Stop,0.83,0.07,0.18,0.14,0.02,0.06,0.19
Hispanic Arrests Share / Stops Share Per Stop,0.90,0.04,0.11,0.10,0.02,0.05,0.14
Native American Arrests Share / Stops Share Per Stop,0.71,0.05,0.13,0.09,0.04,0.05,0.15
Other Arrests Share / Stops Share Per Stop,0.65,0.07,0.23,0.17,0.04,0.09,0.24
White Arrests Share / Stops Share Per Stop,0.73,0.07,0.19,0.16,0.03,0.07,0.21


In [62]:
create_corr_matrix(total_citations_df, arrests_share_div_stops_total_df, "Arrests Share / Stops Total", "Total Citations", "Arrested")

Correlation Matrix: Total Citations vs. Arrested,Total Citations,Asian Citations,Black Citations,Hispanic Citations,Native American Citations,Other Citations,White Citations
Total Arrests Share / Stops Total Per Stop,0.89,-0.01,-0.02,-0.02,0.00,-0.01,-0.02
Asian Arrests Share / Stops Total Per Stop,0.71,-0.00,-0.03,-0.03,-0.00,-0.01,-0.03
Black Arrests Share / Stops Total Per Stop,0.83,-0.00,-0.02,-0.02,-0.00,-0.01,-0.02
Hispanic Arrests Share / Stops Total Per Stop,0.90,-0.00,-0.02,-0.01,0.00,-0.01,-0.01
Native American Arrests Share / Stops Total Per Stop,0.71,-0.00,-0.02,-0.02,0.05,-0.00,-0.02
Other Arrests Share / Stops Total Per Stop,0.65,-0.01,-0.03,-0.02,0.00,-0.00,-0.03
White Arrests Share / Stops Total Per Stop,0.73,-0.01,-0.03,-0.02,0.00,-0.01,-0.02


In [63]:
create_corr_matrix(total_citations_df, arrests_total_div_stops_share_df, "Arrests Total / Stops Share", "Total Citations", "Arrested")

Correlation Matrix: Total Citations vs. Arrested,Total Citations,Asian Citations,Black Citations,Hispanic Citations,Native American Citations,Other Citations,White Citations
Total Arrests Total / Stops Share Per Stop,0.89,0.77,0.88,0.85,0.38,0.77,0.87
Asian Arrests Total / Stops Share Per Stop,0.71,0.64,0.73,0.68,0.38,0.65,0.70
Black Arrests Total / Stops Share Per Stop,0.83,0.73,0.84,0.80,0.27,0.69,0.78
Hispanic Arrests Total / Stops Share Per Stop,0.90,0.76,0.86,0.85,0.35,0.76,0.90
Native American Arrests Total / Stops Share Per Stop,0.71,0.63,0.68,0.69,0.28,0.55,0.69
Other Arrests Total / Stops Share Per Stop,0.65,0.56,0.68,0.62,0.40,0.61,0.64
White Arrests Total / Stops Share Per Stop,0.73,0.66,0.74,0.69,0.44,0.68,0.73


In [64]:
create_corr_matrix(total_citations_df, arrests_total_div_stops_total_df, "Arrests Total / Stops Total", "Total Citations", "Arrested")

Correlation Matrix: Total Citations vs. Arrested,Total Citations,Asian Citations,Black Citations,Hispanic Citations,Native American Citations,Other Citations,White Citations
Total Arrests Total / Stops Total Per Stop,0.89,0.10,0.04,0.07,0.07,0.05,0.05
Asian Arrests Total / Stops Total Per Stop,0.71,0.09,0.03,0.06,0.06,0.05,0.04
Black Arrests Total / Stops Total Per Stop,0.83,0.10,0.04,0.07,0.05,0.05,0.05
Hispanic Arrests Total / Stops Total Per Stop,0.90,0.09,0.04,0.08,0.06,0.05,0.05
Native American Arrests Total / Stops Total Per Stop,0.71,0.07,0.03,0.05,0.11,0.03,0.03
Other Arrests Total / Stops Total Per Stop,0.65,0.06,0.02,0.04,0.07,0.04,0.03
White Arrests Total / Stops Total Per Stop,0.73,0.08,0.03,0.06,0.08,0.05,0.04


In [81]:
create_corr_matrix(pct_stopped_df_2, arrests_div_stops_df, "Arrests / Stops", "Percent Stops", "Arrested")

Correlation Matrix: Percent Stops vs. Arrested,Total as Percent of Stopped,Asian as Percent of Stopped,Black as Percent of Stopped,Hispanic as Percent of Stopped,Native American as Percent of Stopped,Other as Percent of Stopped,White as Percent of Stopped
Total Arrests / Stops,nan,nan,nan,nan,nan,nan,nan
Asian Arrests / Stops,-0.01,0.02,-0.03,-0.02,-0.00,0.02,-0.02
Black Arrests / Stops,0.08,0.04,0.18,0.01,0.01,0.02,0.00
Hispanic Arrests / Stops,0.06,0.05,0.01,0.15,0.02,0.02,-0.01
Native American Arrests / Stops,-0.00,-0.00,-0.01,-0.01,0.01,0.01,-0.01
Other Arrests / Stops,-0.00,-0.00,-0.03,0.01,0.01,0.01,-0.02
White Arrests / Stops,-0.10,-0.07,-0.12,-0.10,-0.02,-0.04,0.02


In [82]:
create_corr_matrix(arrests_total_div_stops_total_df, arrests_share_div_stops_share_df, "% arrested / % stopped", "arrested / stopped", "arrested")

Correlation Matrix: arrested / stopped vs. arrested,Total Arrested,Asian Arrested Per Stop,Black Arrested Per Stop,Hispanic Arrested Per Stop,Native American Arrested Per Stop,Other Arrested Per Stop,White Arrested Per Stop
Total % arrested / % stopped,1.00,0.05,0.10,0.09,0.02,0.04,0.14
Asian % arrested / % stopped,0.13,0.56,0.03,0.02,-0.00,0.04,0.02
Black % arrested / % stopped,0.07,0.00,0.33,0.00,0.01,-0.00,0.00
Hispanic % arrested / % stopped,0.11,0.00,0.01,0.45,-0.00,0.00,0.01
Native American % arrested / % stopped,0.07,-0.00,0.06,0.00,0.78,0.00,0.00
Other % arrested / % stopped,0.07,0.03,0.01,0.01,0.00,0.65,0.02
White % arrested / % stopped,0.08,-0.00,-0.01,-0.00,-0.00,0.00,0.36


In [33]:
create_corr_matrix(citations_df, arrests_df, "Arrested", "Citations", "Arrests")

Correlation Matrix: Citations vs. Arrests,Total Citations,Asian Citations,Black Citations,Hispanic Citations,Native American Citations,Other Citations,White Citations
Total Arrested Per Stop,nan,0.07,0.18,0.14,0.02,0.07,0.20
Asian Arrested Per Stop,-0.01,0.03,0.02,0.02,0.00,0.00,0.01
Black Arrested Per Stop,0.07,0.05,0.16,0.07,0.00,0.04,0.12
Hispanic Arrested Per Stop,0.09,0.03,0.12,0.15,0.01,0.05,0.12
Native American Arrested Per Stop,-0.00,0.00,-0.00,-0.01,0.02,0.04,-0.01
Other Arrested Per Stop,-0.01,0.01,0.01,0.01,0.01,0.04,0.01
White Arrested Per Stop,-0.06,-0.03,-0.08,-0.06,-0.00,-0.03,-0.04


In [29]:
create_corr_matrix(pct_stopped_df, arrests_df, "Arrested", " ", "Arrests")

/opt/conda/lib/python3.7/site-packages/matplotlib/colors.py:527: RuntimeWarning:

invalid value encountered in less



Correlation Matrix: vs. Arrests,Total as Percent of Stopped,Asian as Percent of Stopped,Black as Percent of Stopped,Hispanic as Percent of Stopped,Native American as Percent of Stopped,Other as Percent of Stopped,White as Percent of Stopped
Total Arrested Per Stop,nan,nan,nan,nan,nan,nan,nan
Asian Arrested Per Stop,-0.01,0.01,0.02,0.02,0.01,0.03,0.02
Black Arrested Per Stop,0.08,0.03,0.10,0.06,-0.00,0.03,0.09
Hispanic Arrested Per Stop,0.06,0.03,0.07,0.06,0.00,0.04,0.07
Native American Arrested Per Stop,-0.00,-0.00,-0.01,-0.01,0.00,0.00,-0.01
Other Arrested Per Stop,-0.00,0.01,0.02,0.02,0.00,0.01,0.02
White Arrested Per Stop,-0.10,-0.05,-0.14,-0.10,-0.01,-0.06,-0.13


In [23]:
create_corr_matrix(citations_df, probable_cause_df_2, "Probable Cause Searches", "Citations", "Probable Cause Searches")

Correlation Matrix: Citations vs. Probable Cause Searches,Total Citations,Asian Citations,Black Citations,Hispanic Citations,Native American Citations,Other Citations,White Citations
Total Probable Cause Searches Per Stop,nan,-0.00,-0.02,-0.01,-0.00,-0.01,-0.02
Asian Probable Cause Searches Per Stop,-0.03,0.02,-0.02,-0.01,-0.01,-0.01,-0.02
Black Probable Cause Searches Per Stop,0.02,0.01,0.08,-0.02,0.00,-0.02,-0.03
Hispanic Probable Cause Searches Per Stop,-0.00,-0.00,0.02,0.03,0.03,-0.01,-0.04
Native American Probable Cause Searches Per Stop,0.01,0.01,-0.00,-0.00,0.09,0.19,0.00
Other Probable Cause Searches Per Stop,-0.03,0.00,-0.02,-0.01,0.00,0.03,-0.03
White Probable Cause Searches Per Stop,-0.03,0.01,-0.05,-0.03,-0.02,-0.02,0.02


In [16]:
create_corr_matrix(pct_stopped_df, probable_cause_df_2, "Probable Cause Searches", "Pct Stopped", "Stops")

Correlation Matrix: Pct Stopped vs. Stops,Total as Percent of Stopped,Asian as Percent of Stopped,Black as Percent of Stopped,Hispanic as Percent of Stopped,Native American as Percent of Stopped,Other as Percent of Stopped,White as Percent of Stopped
Total Probable Cause Searches Per Stop,nan,nan,nan,nan,nan,nan,nan
Asian Probable Cause Searches Per Stop,-0.04,0.02,-0.02,-0.01,-0.01,-0.01,-0.03
Black Probable Cause Searches Per Stop,0.02,0.01,0.10,-0.01,0.01,-0.02,-0.05
Hispanic Probable Cause Searches Per Stop,0.01,-0.01,0.01,0.10,0.02,0.00,-0.04
Native American Probable Cause Searches Per Stop,-0.00,0.00,-0.00,-0.00,0.02,0.02,-0.00
Other Probable Cause Searches Per Stop,-0.03,0.02,-0.02,-0.01,0.01,0.03,-0.03
White Probable Cause Searches Per Stop,0.00,-0.02,-0.07,-0.05,-0.02,0.00,0.09


In [31]:
create_corr_matrix_2(raw_total_stops, probable_cause_df, "Pct Stopped", "Probable Cause Searches")

/opt/conda/lib/python3.7/site-packages/scipy/stats/stats.py:3508: PearsonRConstantInputWarning:

An input array is constant; the correlation coefficent is not defined.

/opt/conda/lib/python3.7/site-packages/matplotlib/colors.py:527: RuntimeWarning:

invalid value encountered in less



Correlation Matrix: Pct Stopped vs. Probable Cause Searches,Total as Percent of Stopped,Asian as Percent of Stopped,Black as Percent of Stopped,Hispanic as Percent of Stopped,Native American as Percent of Stopped,Other as Percent of Stopped,White as Percent of Stopped
Total Probable Cause Searches,nan,nan,nan,nan,nan,nan,nan
Asian Probable Cause Searches,0.57,-0.42,-0.25,-0.31,-0.67,-0.38,-0.37
Black Probable Cause Searches,0.82,-0.38,-0.12,-0.36,-0.61,-0.41,-0.43
Hispanic Probable Cause Searches,0.76,-0.39,-0.23,-0.26,-0.67,-0.37,-0.40
Native American Probable Cause Searches,0.64,-0.27,-0.06,-0.24,-0.52,-0.30,-0.28
Other Probable Cause Searches,0.66,-0.44,-0.23,-0.34,-0.55,-0.43,-0.42
White Probable Cause Searches,0.51,-0.38,-0.27,-0.32,-0.67,-0.34,-0.33


In [35]:
# create_corr_matrix(searches_df, pct_stopped_df, "Searched", "Percent of Stops", "Searches")

Correlation Matrix: Percent of Stops vs. Searches,Total Searched Per Stop,Asian Searched Per Stop,Black Searched Per Stop,Hispanic Searched Per Stop,Native American Searched Per Stop,Other Searched Per Stop,White Searched Per Stop
Total Searched Per Stop,nan,-0.06,0.08,0.07,0.00,-0.05,-0.06
Asian Searched Per Stop,nan,0.02,0.03,0.02,-0.00,0.01,-0.05
Black Searched Per Stop,nan,0.03,0.10,0.07,-0.01,0.02,-0.14
Hispanic Searched Per Stop,nan,0.02,0.06,0.07,-0.01,0.03,-0.10
Native American Searched Per Stop,nan,0.00,0.03,0.01,0.00,0.00,-0.03
Other Searched Per Stop,nan,0.04,0.01,0.02,-0.00,0.01,-0.04
White Searched Per Stop,nan,0.02,0.08,0.07,-0.00,0.01,-0.12


In [36]:
create_corr_matrix(arrests_df, pct_stopped_df, "Arrested", "Percent of Stops", "Arrested")

Correlation Matrix: Percent of Stops vs. Arrested,Total Arrested,Asian Arrested Per Stop,Black Arrested Per Stop,Hispanic Arrested Per Stop,Native American Arrested Per Stop,Other Arrested Per Stop,White Arrested Per Stop
Total Arrested Per Stop,nan,-0.01,0.08,0.06,-0.00,-0.00,-0.10
Asian Arrested Per Stop,nan,0.01,0.03,0.03,-0.00,0.01,-0.05
Black Arrested Per Stop,nan,0.02,0.10,0.07,-0.01,0.02,-0.14
Hispanic Arrested Per Stop,nan,0.02,0.06,0.06,-0.01,0.02,-0.10
Native American Arrested Per Stop,nan,0.01,-0.00,0.00,0.00,0.00,-0.01
Other Arrested Per Stop,nan,0.03,0.03,0.04,0.00,0.01,-0.06
White Arrested Per Stop,nan,0.02,0.09,0.07,-0.01,0.02,-0.13


In [38]:
create_corr_matrix(pct_stopped_df_2, searches_df, "Searched", "Percent of Stops", "Searches")

Correlation Matrix: Percent of Stops vs. Searches,Total as Percent of Stopped 2,Asian as Percent of Stopped 2,Black as Percent of Stopped 2,Hispanic as Percent of Stopped 2,Native American as Percent of Stopped 2,Other as Percent of Stopped 2,White as Percent of Stopped 2
Total Searched Per Stop,nan,nan,nan,nan,nan,nan,nan
Asian Searched Per Stop,-0.06,0.05,-0.04,-0.02,-0.01,0.02,-0.04
Black Searched Per Stop,0.08,0.04,0.19,0.03,0.04,0.01,-0.02
Hispanic Searched Per Stop,0.07,0.05,0.03,0.16,0.02,0.02,-0.01
Native American Searched Per Stop,0.00,-0.01,-0.01,-0.01,0.01,0.00,0.01
Other Searched Per Stop,-0.05,-0.01,-0.03,0.01,0.00,0.04,-0.04
White Searched Per Stop,-0.06,-0.07,-0.14,-0.12,-0.05,-0.04,0.05


In [39]:
create_corr_matrix(citations_df, arrests_df, "Arrested", "Citations", "Arrests")

Correlation Matrix: Citations vs. Arrests,Total Citations,Asian Citations,Black Citations,Hispanic Citations,Native American Citations,Other Citations,White Citations
Total Arrested Per Stop,0.89,0.07,0.18,0.14,0.02,0.07,0.20
Asian Arrested Per Stop,-0.01,0.03,0.02,0.02,0.00,0.00,0.01
Black Arrested Per Stop,0.07,0.05,0.16,0.07,0.00,0.04,0.12
Hispanic Arrested Per Stop,0.09,0.03,0.12,0.15,0.01,0.05,0.12
Native American Arrested Per Stop,-0.00,0.00,-0.00,-0.01,0.02,0.04,-0.01
Other Arrested Per Stop,-0.01,0.01,0.01,0.01,0.01,0.04,0.01
White Arrested Per Stop,-0.06,-0.03,-0.08,-0.06,-0.00,-0.03,-0.04


In [40]:
create_corr_matrix(citations_df, searches_df, "Searched", "Citations", "Searches")

Correlation Matrix: Citations vs. Searches,Total Citations,Asian Citations,Black Citations,Hispanic Citations,Native American Citations,Other Citations,White Citations
Total Searched Per Stop,0.02,0.08,0.04,0.08,0.08,0.04,0.03
Asian Searched Per Stop,-0.06,0.03,-0.04,-0.02,-0.01,-0.01,-0.05
Black Searched Per Stop,0.07,0.04,0.18,0.02,0.04,0.01,0.00
Hispanic Searched Per Stop,0.05,0.07,0.05,0.20,0.02,0.02,-0.01
Native American Searched Per Stop,-0.00,-0.00,-0.01,-0.01,0.03,0.08,-0.01
Other Searched Per Stop,-0.04,-0.01,-0.04,-0.00,0.00,0.10,-0.04
White Searched Per Stop,-0.07,-0.04,-0.08,-0.08,-0.03,-0.05,0.02


In [20]:
create_corr_matrix(citations_df, probable_cause_df, "Searched w/ Prob. Cause", "Citations", "Probable Cause Searches")

Correlation Matrix: Citations vs. Probable Cause Searches,Total Citations,Asian Citations,Black Citations,Hispanic Citations,Native American Citations,Other Citations,White Citations
Total Searched w/ Prob. Cause Per Stop,nan,0.07,0.27,0.19,0.01,0.06,0.11
Asian Searched w/ Prob. Cause Per Stop,-0.01,0.03,0.01,-0.00,-0.01,0.00,0.02
Black Searched w/ Prob. Cause Per Stop,0.15,0.03,0.28,0.08,0.01,0.02,0.06
Hispanic Searched w/ Prob. Cause Per Stop,0.14,0.01,0.14,0.20,0.04,0.04,0.03
Native American Searched w/ Prob. Cause Per Stop,-0.00,0.00,-0.00,-0.00,0.07,0.10,-0.00
Other Searched w/ Prob. Cause Per Stop,-0.01,0.01,0.00,0.00,0.00,0.05,0.00
White Searched w/ Prob. Cause Per Stop,-0.11,-0.00,-0.15,-0.10,-0.02,-0.02,0.04


In [16]:
# create_corr_matrix(accidents_df, searches_per_accident_df, "Searches Per Accident", "Accidents", "Searches Per Accident")
# create_corr_matrix(df1, df2, df2_var, df1_name, df2_name):

df1 = accidents_df
df2 = searches_per_accident_df
df2_var = "Searches Per Accident"
df1_name = "Accidents"
df2_name = "Searches Per Accident"

test = np.zeros((7,7))
for i, col in enumerate(test):
    for j, row in enumerate(test[i]):
        test[i][j] = round(pearsonr(df1.iloc[:, i], df2.iloc[:, j])[0], 2)


corr_df = pd.DataFrame(test)

corr_df.columns = df1.columns

corr_df.rename({0:f'Total {df2_var} Per Stop', 1: f'Asian {df2_var} Per Stop', 2: f'Black {df2_var} Per Stop', 3: f'Hispanic {df2_var} Per Stop', 4: f'Native American {df2_var} Per Stop', 5: f'Other {df2_var} Per Stop', 6: f'White {df2_var} Per Stop'}, inplace=True)
corr_df['Total Accidents'].iloc[0] = np.nan
corr_df['Native American Accidents'].iloc[0] = 0.00


import seaborn as sns
cm = sns.light_palette("blue", as_cmap=True)


with pd.option_context('display.precision', 2):
    html1 = (corr_df.style
            .background_gradient(cmap=cm, axis=None)
           )

html1.columns.name = f'Correlation Matrix: {df1_name} vs. {df2_name}'
html1

# del (df1, df2, df2_var, df1_name, df2_name)

Correlation Matrix: Accidents vs. Searches Per Accident,Total Accidents,Asian Accidents,Black Accidents,Hispanic Accidents,Native American Accidents,Other Accidents,White Accidents
Total Searches Per Accident Per Stop,nan,0.32,0.35,0.35,0.00,0.30,0.31
Asian Searches Per Accident Per Stop,0.09,0.10,0.11,0.05,-0.00,0.04,0.14
Black Searches Per Accident Per Stop,0.29,0.06,0.20,0.18,0.01,0.07,0.18
Hispanic Searches Per Accident Per Stop,0.25,0.06,0.18,0.20,0.03,0.09,0.18
Native American Searches Per Accident Per Stop,0.01,-0.00,0.03,0.04,0.00,-0.00,0.03
Other Searches Per Accident Per Stop,0.11,0.05,0.09,0.05,-0.00,0.09,0.10
White Searches Per Accident Per Stop,0.16,0.07,0.18,0.13,0.00,0.08,0.29


In [19]:
# create_corr_matrix(citations_df, probable_cause_df, "Searched w/ Prob. Cause", "Citations", "Probable Cause Searches")

def create_corr_matrix2(df1, df2, df2_var, df1_name, df2_name):
    test = np.zeros((7,7))
    for i, col in enumerate(test):
        for j, row in enumerate(test[i]):
            test[i][j] = round(pearsonr(df1.iloc[:, i], df2.iloc[:, j])[0], 2)


    corr_df = pd.DataFrame(test)

    corr_df.columns = df1.columns
    
    corr_df.rename({0:f'Total {df2_var} Per Stop', 1: f'Asian {df2_var} Per Stop', 2: f'Black {df2_var} Per Stop', 3: f'Hispanic {df2_var} Per Stop', 4: f'Native American {df2_var} Per Stop', 5: f'Other {df2_var} Per Stop', 6: f'White {df2_var} Per Stop'}, inplace=True)
#     corr_df['Total Accidents'].iloc[0] = np.nan


    import seaborn as sns
    cm = sns.light_palette("blue", as_cmap=True)


    with pd.option_context('display.precision', 2):
        html1 = (corr_df.style
                .background_gradient(cmap=cm, axis=None)
               )

    html1.columns.name = f'Correlation Matrix: {df1_name} vs. {df2_name}'
    return(html1)

In [20]:
create_corr_matrix2(accidents_df, searches_df, "Searched", "Accidents", "Searches")

Correlation Matrix: Accidents vs. Searches,Total Accidents,Asian Accidents,Black Accidents,Hispanic Accidents,Native American Accidents,Other Accidents,White Accidents
Total Searched Per Stop,0.01,0.09,0.19,0.15,0.03,0.06,0.19
Asian Searched Per Stop,-0.03,0.04,0.08,0.08,0.03,0.02,0.10
Black Searched Per Stop,0.00,0.06,0.17,0.15,0.04,0.07,0.16
Hispanic Searched Per Stop,0.01,0.08,0.15,0.13,0.03,0.04,0.13
Native American Searched Per Stop,-0.01,0.00,0.02,0.02,-0.00,-0.01,-0.00
Other Searched Per Stop,-0.02,0.05,0.12,0.05,0.00,0.01,0.07
White Searched Per Stop,-0.06,0.07,0.16,0.11,0.01,0.05,0.20


In [10]:
race_set = {'ASIAN', 'BLACK', 'HISPANIC', 'NATIVE AMERICAN', 'OTHER', 'WHITE'}

In [11]:


# gdf = geopandas.GeoDataFrame(
#     df2, geometry=geopandas.points_from_xy(df2.Longitude, df2.Latitude))   
     
lat = []
lon = []
for x in coordinates_dict:
    for y in coordinates_dict[x]:
        lon.append(x)
        lat.append(y)



df2 = pd.DataFrame(
    {'Total Citations': [coordinates_dict[x][y][6] for x in coordinates_dict for y in coordinates_dict[x]],
     'Asian Citations': [coordinates_dict[x][y][0] for x in coordinates_dict for y in coordinates_dict[x]],
     'Black Citations': [coordinates_dict[x][y][1] for x in coordinates_dict for y in coordinates_dict[x]],
     'Hispanic Citations': [coordinates_dict[x][y][2] for x in coordinates_dict for y in coordinates_dict[x]],
     'Native American Citations': [coordinates_dict[x][y][3] for x in coordinates_dict for y in coordinates_dict[x]],
     'Other Citations': [coordinates_dict[x][y][4] for x in coordinates_dict for y in coordinates_dict[x]],
     'White Citations': [coordinates_dict[x][y][5] for x in coordinates_dict for y in coordinates_dict[x]],
     
     'Latitude': lat,
     'Longitude': lon,
     
    }
)


gdf = gpd.GeoDataFrame(
    df2, geometry=gpd.points_from_xy(lat,lon))



KeyError: 6

In [ ]:
racial_stops_by_geo = []

for i in range(len(df)):    
    try:
        racial_stops_by_geo.append(coordinates_dict[rounder(df["Longitude"].iloc[i])][rounder(df["Latitude"].iloc[i])][race_key_dict[df["Race"].iloc[i]]+7])
    except:
        racial_stops_by_geo.append(np.nan)

In [ ]:
coordinates_dict[-77][39][7:]

In [ ]:
racial_stops_by_geo2 = []

for i in range(len(df)):
    try:
        racial_stops_by_geo2.append(coordinates_dict[rounder(df["Longitude"].iloc[i])][rounder(df["Latitude"].iloc[i])][7:])
    except:
        racial_stops_by_geo2.append([np.nan]*6)

In [ ]:
# asian_as_local_share = [x[0] for x in racial_stops_by_geo2]
# black_as_local_share = [x[1] for x in racial_stops_by_geo2]
# hispanic_as_local_share = [x[2] for x in racial_stops_by_geo2]
# nativeamerican_as_local_share = [x[3] for x in racial_stops_by_geo2]
# other_as_local_share = [x[4] for x in racial_stops_by_geo2]
# white_as_local_share = [x[5] for x in racial_stops_by_geo2]
# own_as_local_share = 

In [ ]:
# "Race Stops % of local" represents the share of local stops that a racial group accounts for
df["Own Stops % of local"] = racial_stops_by_geo
df["Asian Stops % of local"] = [x[0] for x in racial_stops_by_geo2]
df["Black Stops % of local"] = [x[1] for x in racial_stops_by_geo2]
df["Hispanic Stops % of local"] = [x[2] for x in racial_stops_by_geo2]
df["Native American Stops % of local"] = [x[3] for x in racial_stops_by_geo2]
df["Other Stops % of local"] = [x[4] for x in racial_stops_by_geo2]
df["White Stops % of local"] = [x[5] for x in racial_stops_by_geo2]

In [ ]:
df5.to_csv(r'FINAL GEO CSV 2.csv')

In [ ]:
df.head()

In [ ]:
seqnum = list(range(1,len(df)+1))

In [ ]:
df["seqnum"] = seqnum

In [ ]:
len(df)

In [ ]:
import dtale

In [ ]:
dtale.show(df5)